In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog, font as tk_font
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import threading
import csv
import meep as mp
from scipy.constants import c, k, epsilon_0, mu_0 # c is speed of light, k is Boltzmann constant, epsilon_0, mu_0 for material properties
import logging

# ===================================================================
# DEFINITIVE FIX: Suppress the verbose font finding warnings from matplotlib.
# This prevents the console from being spammed with "findfont" messages.
# ===================================================================
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)


# --- Global Constants for Coordinate System Alignment ---
Y_ANTENNA_PLANE = 0.0 
MEEP_UNIT_LENGTH_M = 1e-3

class SkinCancerSimulator(tk.Tk):
    def __init__(self):
        super().__init__()
        print("ULTRA-DEBUG: SkinCancerSimulator __init__ started.")
        self.title("Definitive MIMO Imaging Suite with Advanced Analytical Testbed")
        self.geometry("2000x1200")

        self.setup_styles()

        # --- Initialize Tkinter BooleanVar/StringVar attributes EARLY ---
        self.auto_spacing_var = tk.BooleanVar(value=True)
        self.test_auto_spacing_var = tk.BooleanVar(value=True)
        self.test_layers_enabled_var = tk.BooleanVar(value=True)
        self.test_targets_enabled_var = tk.BooleanVar(value=True)
        self.fw_auto_spacing_var = tk.BooleanVar(value=True)
        self.fw_layers_enabled_var = tk.BooleanVar(value=True)
        self.fw_targets_enabled_var = tk.BooleanVar(value=True)
        self.v2_auto_spacing_var = tk.BooleanVar(value=True)
        self.v2_layers_enabled_var = tk.BooleanVar(value=True)
        self.v2_targets_enabled_var = tk.BooleanVar(value=True)
        self.run_fdtd_with_analytical_var = tk.BooleanVar(value=True)
        self.da_auto_spacing_var = tk.BooleanVar(value=True)
        self.da_layers_enabled_var = tk.BooleanVar(value=True)
        self.pa_layers_enabled_var = tk.BooleanVar(value=True) # PA for Parametric Analysis
        self.pa_simulation_method_var = tk.StringVar(value="Analytical")
        self.pa_analysis_variable_var = tk.StringVar(value="Frequency (GHz)")

        self.notebook = ttk.Notebook(self, style='App.TNotebook')
        self.notebook.pack(expand=True, fill='both', padx=10, pady=10)

        self.tab1 = ttk.Frame(self.notebook, style='App.TFrame')
        self.tab2 = ttk.Frame(self.notebook, style='App.TFrame')
        self.tab3 = ttk.Frame(self.notebook, style='App.TFrame')
        self.tab4 = ttk.Frame(self.notebook, style='App.TFrame')
        self.tab5 = ttk.Frame(self.notebook, style='App.TFrame')
        self.tab6 = ttk.Frame(self.notebook, style='App.TFrame') # New Tab

        self.notebook.add(self.tab1, text='Skin Cancer Imaging (FDTD)')
        self.notebook.add(self.tab2, text='Algorithm Testbed (Analytical)')
        self.notebook.add(self.tab3, text='Full-Wave EM Testbed (FDTD via Meep)')
        self.notebook.add(self.tab4, text='TestbedV2 (Analytical vs FDTD)')
        self.notebook.add(self.tab5, text='Depth Analysis Tool')
        self.notebook.add(self.tab6, text='Parametric Analysis') # New Tab

        # Initialize widget-related attributes to None
        self.spacing_entry = None
        self.test_skin_offset_entry_widget = None
        self.test_num_layers_entry_widget = None
        self.test_num_targets_entry_widget = None
        self.test_layer_params_frame = None
        self.test_target_params_frame = None
        self.testbed_progress_bar = None
        self.fw_skin_offset_entry_widget = None
        self.fw_num_layers_entry_widget = None
        self.fw_num_targets_entry_widget = None
        self.fw_layer_params_frame = None
        self.fw_target_params_frame = None
        self.fw_testbed_progress_bar = None
        self.v2_skin_offset_entry_widget = None
        self.v2_num_layers_entry_widget = None
        self.v2_num_targets_entry_widget = None
        self.v2_layer_params_frame = None
        self.v2_target_params_frame = None
        self.v2_testbed_progress_bar = None
        self.v2_spacing_entry = None
        self.v2_freq_var = None
        self.da_skin_offset_entry_widget = None
        self.da_num_layers_entry_widget = None
        self.da_layer_params_frame = None
        self.da_min_freq_var = None
        self.da_max_freq_var = None
        self.da_center_freq_display_var = None
        self.da_bandwidth_display_var = None
        self.da_spacing_entry = None
        self.da_antenna_offset_x_entry_widget = None
        self.pa_skin_offset_entry_widget = None
        self.pa_num_layers_entry_widget = None
        self.pa_layer_params_frame = None
        self.pa_variable_min_entry = None
        self.pa_variable_max_entry = None
        self.pa_variable_steps_entry = None
        self.pa_variable_label = None


        self.simulation_thread, self.simulation_results, self.last_run_data = None, None, None
        self.testbed_thread, self.testbed_results = None, None
        self.full_wave_testbed_thread, self.full_wave_testbed_results = None, None
        self.testbed_v2_thread, self.testbed_v2_results = None, None
        self.da_thread, self.da_results = None, None
        self.pa_thread, self.pa_results = None, None # New Thread/Results

        # Initialize colorbar attributes to None
        self.cbar_mimo = None
        self.cbar_das = None
        self.cbar_test_das_2d = None
        self.cbar_fw_das_2d = None
        self.cbar_v2_das_analytical = None
        self.cbar_v2_das_fdtd = None
        self.cbar_pa_heatmap = None # New colorbar

        self.create_skin_imaging_tab()
        self.create_algorithm_testbed_tab()
        self.create_full_wave_em_testbed_tab()
        self.create_testbed_v2_fdtd_tab()
        self.create_depth_analysis_tab()
        self.create_parametric_analysis_tab() # New Tab
        print("ULTRA-DEBUG: SkinCancerSimulator __init__ completed.")
        
    def setup_styles(self):
        """UI UPGRADE: Configures a professional light theme for the application."""
        self.style = ttk.Style(self)
        self.style.theme_use('clam')

        self.BG_COLOR = "#F0F0F0"
        self.FRAME_COLOR = "#FFFFFF"
        self.TEXT_COLOR = "#1F1F1F"
        self.PRIMARY_COLOR = "#007ACC"
        self.LIGHT_PRIMARY_COLOR = "#E6F3FF"
        
        self.configure(background=self.BG_COLOR)

        # --- FONT FIX: Programmatically find the best available font ---
        try:
            available_fonts = set(tk_font.families())
            preferred_fonts = ['Segoe UI', 'DejaVu Sans', 'Calibri', 'Arial', 'Helvetica', 'sans-serif']
            
            font_family_base = 'sans-serif' # A generic default
            for font in preferred_fonts:
                if font in available_fonts:
                    font_family_base = font
                    break
            
            print(f"DEBUG: Selected '{font_family_base}' as the base GUI font.")
            
            self.option_add("*Font", f"{font_family_base} 11")
            
            self.style.configure('.', background=self.BG_COLOR, foreground=self.TEXT_COLOR, font=(font_family_base, 11))
            self.style.configure('Header.TLabel', font=(font_family_base, 12, "bold"), background=self.BG_COLOR)
            self.style.configure('TLabelframe.Label', background=self.BG_COLOR, foreground=self.TEXT_COLOR, font=(font_family_base, 10, "italic"))

        except Exception as e:
            print(f"Font selection failed: {e}. Falling back to system default.")
            self.option_add("*Font", "sans-serif 11")
            self.style.configure('.', font=('sans-serif', 11))
            self.style.configure('Header.TLabel', font=('sans-serif', 12, 'bold'))
            self.style.configure('TLabelframe.Label', font=('sans-serif', 10, 'italic'))

        # Configure styles for widgets
        self.style.configure('App.TFrame', background=self.BG_COLOR)
        self.style.configure('App.TNotebook', background=self.BG_COLOR, borderwidth=0)
        self.style.configure('App.TNotebook.Tab', background=self.BG_COLOR, foreground=self.TEXT_COLOR, padding=[10, 5], borderwidth=1)
        self.style.map('App.TNotebook.Tab',
                       background=[('selected', self.FRAME_COLOR)],
                       foreground=[('selected', self.PRIMARY_COLOR)])
        
        self.style.configure('TLabel', background=self.BG_COLOR, foreground=self.TEXT_COLOR)
        self.style.configure('TCheckbutton', background=self.BG_COLOR, foreground=self.TEXT_COLOR)
        self.style.configure('TEntry', fieldbackground=self.FRAME_COLOR, foreground=self.TEXT_COLOR, borderwidth=1)
        self.style.configure('TLabelframe', background=self.BG_COLOR, bordercolor="#D0D0D0", foreground=self.TEXT_COLOR)
        self.style.configure('TButton', background=self.PRIMARY_COLOR, foreground='white', borderwidth=1, padding=6)
        self.style.map('TButton',
                       background=[('active', '#005A9E'), ('disabled', '#B0B0B0')])
        self.style.configure('TMenubutton', background=self.FRAME_COLOR, foreground=self.TEXT_COLOR)

        # Configure Matplotlib styles
        plt.style.use('seaborn-v0_8-notebook')
        plt.rcParams.update({
            'figure.facecolor': self.FRAME_COLOR,
            'axes.facecolor': self.FRAME_COLOR,
            'axes.edgecolor': '#B0B0B0',
            'axes.labelcolor': self.TEXT_COLOR,
            'xtick.color': self.TEXT_COLOR,
            'ytick.color': self.TEXT_COLOR,
            'text.color': self.TEXT_COLOR,
            'grid.color': '#D0D0D0',
            'legend.facecolor': self.BG_COLOR,
            'legend.edgecolor': '#D0D0D0',
            'font.family': ['Segoe UI', 'DejaVu Sans', 'Calibri', 'Arial', 'sans-serif']
        })


    def _create_scrollable_param_frame(self, parent, text_label="Parameters", min_canvas_height=0):
        outer_frame = ttk.LabelFrame(parent, text=text_label, padding="10", style='TLabelframe')
        canvas = tk.Canvas(outer_frame, borderwidth=0, background=self.FRAME_COLOR, highlightthickness=0, height=min_canvas_height)
        inner_frame = ttk.Frame(canvas, padding="5", style='App.TFrame')
        vsb = ttk.Scrollbar(outer_frame, orient="vertical", command=canvas.yview)
        canvas.configure(yscrollcommand=vsb.set)
        vsb.pack(side="right", fill="y")
        canvas.pack(side="left", fill="both", expand=True)
        canvas_window = canvas.create_window((0, 0), window=inner_frame, anchor="nw")

        def on_frame_configure(event):
            canvas.configure(scrollregion=canvas.bbox("all"))
        
        def on_canvas_configure(event):
            canvas.itemconfig(canvas_window, width=event.width)

        inner_frame.bind("<Configure>", on_frame_configure)
        canvas.bind("<Configure>", on_canvas_configure)
        
        def on_mousewheel(event):
            canvas.yview_scroll(int(-1*(event.delta/120)), "units")
            
        canvas.bind_all("<MouseWheel>", on_mousewheel)

        return outer_frame, inner_frame

    # ===================================================================
    # TAB 1: SKIN CANCER IMAGING 
    # ===================================================================
    def create_skin_imaging_tab(self):
        print("ULTRA-DEBUG: create_skin_imaging_tab started.")
        main_frame = ttk.Frame(self.tab1, padding="10", style='App.TFrame')
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        control_outer_frame, control_frame = self._create_scrollable_param_frame(main_frame, text_label="Simulation Parameters")
        control_outer_frame.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)
        
        plot_frame = ttk.Frame(main_frame, padding="10", style='App.TFrame')
        plot_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        self.params = {}
        ttk.Label(control_frame, text="--- MIMO Antenna Array ---", style='Header.TLabel').pack(pady=5, anchor='w')
        self.add_param(control_frame, self.params, "num_tx", "Num Transmitters:", 4)
        self.add_param(control_frame, self.params, "num_rx", "Num Receivers:", 16)
        self.add_param(control_frame, self.params, "freq_max", "Center Frequency (GHz):", 77)
        spacing_frame = ttk.Frame(control_frame); spacing_frame.pack(fill=tk.X, pady=2)
        ttk.Checkbutton(spacing_frame, text="Auto-calculate spacing (λ/2)", variable=self.auto_spacing_var, command=self.toggle_spacing_entry).pack(anchor='w', padx=5)
        returned_widgets_spacing = self.add_param(control_frame, self.params, "antenna_spacing", "Antenna Spacing (mm):", 1.95, return_var_and_widget=True)
        self.spacing_entry = returned_widgets_spacing['widget']
        
        ttk.Label(control_frame, text="--- Tumor & Skin ---", style='Header.TLabel').pack(pady=10, anchor='w')
        tumor_presence_frame = ttk.Frame(control_frame); tumor_presence_frame.pack(fill=tk.X, pady=2, padx=5)
        ttk.Label(tumor_presence_frame, text="Tumor Presence:", width=22).pack(side=tk.LEFT)
        self.tumor_present_var = tk.StringVar(value="Present (Unhealthy)")
        ttk.OptionMenu(tumor_presence_frame, self.tumor_present_var, "Present (Unhealthy)", "Present (Unhealthy)", "Absent (Healthy)").pack(side=tk.RIGHT, expand=True, fill=tk.X)
        self.add_param(control_frame, self.params, "tumor_x", "Tumor X Position (mm):", 2)
        self.add_param(control_frame, self.params, "tumor_y", "Tumor Y Position (mm):", -4)
        self.add_param(control_frame, self.params, "tumor_radius", "Tumor Radius (mm):", 0.75)
        self.add_param(control_frame, self.params, "tumor_permit", "Tumor Permittivity (εᵣ):", 60)
        self.add_param(control_frame, self.params, "tumor_conductivity", "Tumor Conductivity (S/m):", 0.1) 
        
        self.add_param(control_frame, self.params, "skin_surface_offset", "Skin Surface Offset (mm):", 2.5) 
        self.layer_entries = []
        num_layers_var_wrapper = self.add_param(control_frame, self.params, "num_layers", "Number of Skin Layers:", 2, return_var_and_widget=True)
        num_layers_var = num_layers_var_wrapper['var']
        
        outer_layer_scroll_frame_tab1, self.layer_params_frame = self._create_scrollable_param_frame(control_frame, text_label="Skin Layer Properties", min_canvas_height=200)
        outer_layer_scroll_frame_tab1.pack(fill=tk.X, expand=True, pady=5, padx=5)
        self._create_dynamic_layer_widgets(self.layer_params_frame, num_layers_var, self.layer_entries) 
        
        ttk.Label(control_frame, text="--- Simulation Control ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(control_frame, self.params, "resolution", "Sim Resolution (pixels/mm):", 15)
        self.add_param(control_frame, self.params, "pml_thickness", "PML Thickness (mm):", 2.0) 

        self.run_button = ttk.Button(control_frame, text="Run FDTD Simulation", command=self.run_skin_simulation)
        self.run_button.pack(pady=10, fill=tk.X, padx=5)
        self.save_button = ttk.Button(control_frame, text="Save MIMO Matrix to CSV", command=self.save_to_csv, state='disabled')
        self.save_button.pack(pady=5, fill=tk.X, padx=5)
        self.progress_bar = ttk.Progressbar(control_frame, mode='determinate'); self.progress_bar.pack(pady=5, fill=tk.X, padx=5)
        self.status_label = ttk.Label(control_frame, text="Status: Idle"); self.status_label.pack(pady=5, fill=tk.X, padx=5)
        self.toggle_spacing_entry()

        self.fig, self.axes = plt.subplots(3, 2, figsize=(12, 12))
        (self.ax_scenario, self.ax_mimo_matrix), (self.ax_saft_diag, self.ax_saft_img), (self.ax_spectrum, self.ax_placeholder) = self.axes
        self.ax_placeholder.axis('off'); 
        self.fig.tight_layout(pad=4.0, h_pad=6.0)
        self.canvas = FigureCanvasTkAgg(self.fig, master=plot_frame); self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        self.plot_all_empty()
        print("ULTRA-DEBUG: create_skin_imaging_tab completed.")

    def run_skin_simulation(self):
        if self.simulation_thread and self.simulation_thread.is_alive(): return
        self.run_button.config(state='disabled'); self.save_button.config(state='disabled'); self.progress_bar.config(value=0)
        self.simulation_results = None
        self.simulation_thread = threading.Thread(target=self._execute_skin_simulation_thread)
        self.simulation_thread.daemon = True; self.simulation_thread.start()
        self.after(100, self._check_skin_simulation_status)

    def _execute_skin_simulation_thread(self):
        try:
            p = {name: var.get() for name, var in self.params.items()}; p['tumor_present'] = self.tumor_present_var.get() == "Present (Unhealthy)"
            layer_params = self.get_layer_params(self.layer_entries) 
            
            fc = p['freq_max'] * 1e9; wavelength_mm = (c / fc) * 1000
            if self.auto_spacing_var.get(): p['antenna_spacing'] = wavelength_mm / 2; self.params['antenna_spacing'].set(round(p['antenna_spacing'], 4))
            
            mimo_matrix, spectrum_data, freq_axis = self.perform_fdtd_simulations(p, layer_params)
            self.status_label.config(text="Status: Reconstructing Image (SAFT/DAS)...")
            
            epsilon_eff = self.calculate_effective_permittivity(layer_params, abs(p['tumor_y']))
            array_width = (p['num_rx'] - 1) * p['antenna_spacing']
            img_grid_x = np.linspace(-array_width / 1.8, array_width / 1.8, 121); img_grid_y = np.linspace(p['tumor_y'] - 4, -0.5, 121)
            saft_image = self.saft_das_beamforming(mimo_matrix, p, img_grid_x, img_grid_y, epsilon_eff)
            
            self.simulation_results = {"params": p, "layers": layer_params, "mimo_matrix": mimo_matrix, "img_grid_x": img_grid_x, 
                                       "img_grid_y": img_grid_y, "saft_image": saft_image, "spectrum_data": spectrum_data, "freq_axis": freq_axis}
        except Exception as e: 
            import traceback
            self.simulation_results = {"error": f"{e}\n{traceback.format_exc()}"}

    def _check_skin_simulation_status(self):
        if self.simulation_results:
            self.progress_bar.stop(); self.status_label.config(text="Status: Idle"); self.run_button.config(state='normal')
            if "error" in self.simulation_results: messagebox.showerror("Simulation Error", str(self.simulation_results["error"])); return
            self.last_run_data = self.simulation_results; self.save_button.config(state='normal')
            data = self.simulation_results
            self.plot_scenario(data['params'], data['layers']); self.plot_mimo_matrix(data['mimo_matrix'])
            self.plot_saft_diagnostic(data['mimo_matrix'], data['params'], data['layers']); self.plot_saft_image(data['img_grid_x'], data['img_grid_y'], data['saft_image'], data['params'])
            self.plot_spectrum(data['spectrum_data'], data['freq_axis']); self.canvas.draw(); messagebox.showinfo("Complete", "Simulation and analysis complete.")
        else: self.after(100, self._check_skin_simulation_status)

    def perform_fdtd_simulations(self, p, layer_params):
        num_tx, num_rx = int(p['num_tx']), int(p['num_rx']); mimo_matrix = np.zeros((num_tx, num_rx), dtype=np.complex128)
        spectrum_healthy, spectrum_unhealthy, freq_axis = None, None, None
        
        total_skin_thickness_mm = sum(l['thickness'] for l in layer_params)
        skin_surface_offset = p['skin_surface_offset'] 
        
        max_y_depth_from_antenna_plane_mm = 0.0
        if p['tumor_present']:
            max_y_depth_from_antenna_plane_mm = max(max_y_depth_from_antenna_plane_mm, abs(p['tumor_y']) + p['tumor_radius'])
        max_y_depth_from_antenna_plane_mm = max(max_y_depth_from_antenna_plane_mm, total_skin_thickness_mm + skin_surface_offset)
        
        array_max_x_span_mm = (max(p['num_tx'], p['num_rx']) - 1) * p['antenna_spacing']
        max_dist_one_way_mm = np.sqrt((array_max_x_span_mm / 2)**2 + (max_y_depth_from_antenna_plane_mm + 5)**2) 
        
        max_permit_in_domain = max([l['permit'] for l in layer_params] + ([p['tumor_permit']] if p['tumor_present'] else []) + [1.0])
        slowest_speed_m_s = c / np.sqrt(max_permit_in_domain)
        min_physical_travel_time_s = (2 * max_dist_one_way_mm * 1e-3) / slowest_speed_m_s 
        sim_time_s = min_physical_travel_time_s * 1.5 
        run_until_meep = sim_time_s * (c / MEEP_UNIT_LENGTH_M) 

        domain_x = (p['num_rx'] + 8) * p['antenna_spacing']
        domain_y = (p['pml_thickness'] * 2) + skin_surface_offset + total_skin_thickness_mm + max_y_depth_from_antenna_plane_mm + 10 
        
        p['cell_size'] = mp.Vector3(domain_x, domain_y, 0)
        p['pml_layers'] = [mp.PML(thickness=p['pml_thickness'])]
        fcen_ghz = p['freq_max']; fwidth_ghz = p['freq_max'] * 0.4
        
        p['fcen_meep'] = fcen_ghz * 1e9 * MEEP_UNIT_LENGTH_M / c
        p['df_meep'] = fwidth_ghz * 1e9 * MEEP_UNIT_LENGTH_M / c
        
        meep_y_antenna_plane = (domain_y / 2) - p['pml_thickness'] 
        
        tx_pos_x = np.arange(num_tx) * p['antenna_spacing'] - (num_tx - 1) * p['antenna_spacing'] / 2
        
        geometry_healthy = []
        current_y_pos_meep = meep_y_antenna_plane - p['skin_surface_offset'] 
        for layer in layer_params:
            layer_center_y_meep = current_y_pos_meep - layer['thickness'] / 2
            geometry_healthy.append(mp.Block(center=mp.Vector3(0, layer_center_y_meep, 0),
                                            size=mp.Vector3(p['cell_size'].x, layer['thickness'], mp.inf),
                                            material=mp.Medium(epsilon=layer['permit'], 
                                                               D_conductivity=layer['conductivity'] / (2 * np.pi * fcen_ghz * 1e9 * epsilon_0)))) 
            current_y_pos_meep -= layer['thickness']
        
        geometry_unhealthy = list(geometry_healthy)
        if p['tumor_present']:
            tumor_center_y_meep = meep_y_antenna_plane + p['tumor_y'] 
            tumor_center = mp.Vector3(p['tumor_x'], tumor_center_y_meep, 0)
            geometry_unhealthy.append(mp.Cylinder(center=tumor_center, radius=p['tumor_radius'], height=mp.inf, 
                                                material=mp.Medium(epsilon=p['tumor_permit'], 
                                                                    D_conductivity=p['tumor_conductivity'] / (2 * np.pi * fcen_ghz * 1e9 * epsilon_0))))
        
        progress_max = num_tx * 2 if p['tumor_present'] else num_tx; self.progress_bar.config(mode='determinate', maximum=progress_max, value=0)
        num_freq_points = 100
        
        for tx_idx in range(num_tx):
            tx_pos_meep = mp.Vector3(tx_pos_x[tx_idx], meep_y_antenna_plane)
            sources = [mp.Source(src=mp.GaussianSource(frequency=p['fcen_meep'], fwidth=p['df_meep']), component=mp.Ez, center=tx_pos_meep)]
            
            self.status_label.config(text=f"Status: Simulating Healthy (Tx {tx_idx+1}/{num_tx})")
            signals_healthy, spectrum_h, freq_axis = self._run_single_fdtd_sim(
                p, sources, geometry_healthy, run_until_meep, num_freq_points, tx_pos_x, meep_y_antenna_plane
            )
            if tx_idx == num_tx // 2: spectrum_healthy = spectrum_h
            self.progress_bar.step()
            
            if p['tumor_present']:
                self.status_label.config(text=f"Status: Simulating Unhealthy (Tx {tx_idx+1}/{num_tx})")
                signals_unhealthy, spectrum_u, _ = self._run_single_fdtd_sim(
                    p, sources, geometry_unhealthy, run_until_meep, num_freq_points, tx_pos_x, meep_y_antenna_plane
                )
                if tx_idx == num_tx // 2: spectrum_unhealthy = spectrum_u
                mimo_matrix[tx_idx, :] = signals_unhealthy - signals_healthy
            else: mimo_matrix[tx_idx, :] = np.zeros_like(signals_healthy)
            self.progress_bar.step()
        
        return mimo_matrix, {'healthy': spectrum_healthy, 'unhealthy': spectrum_unhealthy}, freq_axis

    def _run_single_fdtd_sim(self, p, sources, geometry, run_until_time_meep, num_freq, tx_pos_x, array_y_pos_meep):
        num_rx, d = int(p['num_rx']), p['antenna_spacing']; cell = p['cell_size']; fcen_meep, df_meep = p['fcen_meep'], p['df_meep']
        rx_pos_x = np.arange(num_rx) * d - (num_rx - 1) * d / 2
        
        sim = mp.Simulation(cell_size=cell, boundary_layers=p['pml_layers'], geometry=geometry, sources=sources, resolution=p['resolution'])
        
        rx_monitors = [sim.add_dft_fields([mp.Ez], fcen_meep, df_meep, num_freq, 
                                           where=mp.Volume(center=mp.Vector3(rx_pos_x[i], array_y_pos_meep, 0),
                                                           size=mp.Vector3(0,0,0))) 
                       for i in range(num_rx)]
        
        sim.run(until=run_until_time_meep); center_freq_index = int(np.floor(num_freq / 2))
        mimo_signals = np.array([sim.get_dft_array(m, mp.Ez, center_freq_index) for m in rx_monitors]).flatten()
        central_rx_idx = num_rx // 2; spectrum_signals = np.array([sim.get_dft_array(rx_monitors[central_rx_idx], mp.Ez, i) for i in range(num_freq)]).flatten()
        
        freq_axis = np.linspace(fcen_meep - df_meep/2, fcen_meep + df_meep/2, num_freq) * (c / MEEP_UNIT_LENGTH_M) / 1e9
        sim.reset_meep(); return mimo_signals, spectrum_signals, freq_axis

    # ===================================================================
    # TAB 2: ALGORITHM TESTBED 
    # ===================================================================
    def create_algorithm_testbed_tab(self):
        print("ULTRA-DEBUG: create_algorithm_testbed_tab started.")
        main_frame = ttk.Frame(self.tab2, padding="5", style='App.TFrame') 
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        control_outer_frame_tab2, scrollable_inner_frame = self._create_scrollable_param_frame(main_frame, text_label="Testbed Parameters")
        control_outer_frame_tab2.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)
        
        plot_frame = ttk.Frame(main_frame, padding="5", style='App.TFrame') 
        plot_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        self.test_params = {}

        ttk.Label(scrollable_inner_frame, text="--- Virtual Array ---", style='Header.TLabel').pack(pady=5, anchor='w')
        self.add_param(scrollable_inner_frame, self.test_params, "test_num_tx_antennas", "Num Transmitters:", 4) 
        self.add_param(scrollable_inner_frame, self.test_params, "test_num_rx_antennas", "Num Receivers:", 16)
        
        test_freq_var = self.add_param(scrollable_inner_frame, self.test_params, "test_freq", "Center Frequency (GHz):", 77, return_var=True)
        spacing_frame = ttk.Frame(scrollable_inner_frame); spacing_frame.pack(fill=tk.X, pady=2)
        ttk.Checkbutton(spacing_frame, text="Auto-calculate spacing (λ/2)", variable=self.test_auto_spacing_var, command=self.toggle_test_spacing_entry).pack(anchor='w', padx=5)
        returned_widgets = self.add_param(scrollable_inner_frame, self.test_params, "test_spacing", "Antenna Spacing (mm):", 1.95, return_var_and_widget=True)
        self.test_spacing_entry = returned_widgets['widget']
        test_freq_var.trace("w", self.update_test_spacing) 
        
        ttk.Label(scrollable_inner_frame, text="--- System Parameters ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(scrollable_inner_frame, self.test_params, "tx_power_dbm", "Tx Power (dBm):", 10.0)
        self.add_param(scrollable_inner_frame, self.test_params, "rx_gain_db", "Rx Gain (dB):", 20.0)
        self.add_param(scrollable_inner_frame, self.test_params, "noise_figure_db", "Noise Figure (dB):", 3.0)
        self.add_param(scrollable_inner_frame, self.test_params, "bandwidth_mhz", "Bandwidth (MHz):", 200.0)
        self.add_param(scrollable_inner_frame, self.test_params, "system_temp_k", "System Temp (K):", 290.0) 

        ttk.Label(scrollable_inner_frame, text="--- Medium Properties ---", style='Header.TLabel').pack(pady=10, anchor='w')
        ttk.Checkbutton(scrollable_inner_frame, text="Enable Skin Layers", variable=self.test_layers_enabled_var, command=self.toggle_test_layer_entries).pack(anchor='w', padx=5)
        test_skin_offset_info = self.add_param(scrollable_inner_frame, self.test_params, "test_skin_surface_offset", "Skin Surface Offset (mm):", 2.5, return_var_and_widget=True) 
        self.test_skin_offset_entry_widget = test_skin_offset_info['widget']
        
        self.test_layer_entries = []
        num_layers_info = self.add_param(scrollable_inner_frame, self.test_params, "test_num_layers", "Num Layers:", 2, return_var_and_widget=True)
        num_layers_var = num_layers_info['var']
        self.test_num_layers_entry_widget = num_layers_info['widget']
        
        outer_layer_scroll_frame_test, self.test_layer_params_frame = self._create_scrollable_param_frame(scrollable_inner_frame, text_label="Skin Layers Properties", min_canvas_height=200)
        outer_layer_scroll_frame_test.pack(fill=tk.X, expand=True, pady=5, padx=5)
        self._create_dynamic_layer_widgets(self.test_layer_params_frame, num_layers_var, self.test_layer_entries) 

        ttk.Label(scrollable_inner_frame, text="--- Multiple Targets ---", style='Header.TLabel').pack(pady=10, anchor='w')
        ttk.Checkbutton(scrollable_inner_frame, text="Enable Multiple Targets", variable=self.test_targets_enabled_var, command=self.toggle_test_target_entries).pack(anchor='w', padx=5)

        self.test_target_entries = []
        num_targets_info = self.add_param(scrollable_inner_frame, self.test_params, "test_num_targets", "Number of Targets:", 1, return_var_and_widget=True)
        num_targets_var = num_targets_info['var']
        self.test_num_targets_entry_widget = num_targets_info['widget']
        
        outer_target_scroll_frame_test, self.test_target_params_frame = self._create_scrollable_param_frame(scrollable_inner_frame, text_label="Target Properties", min_canvas_height=200)
        outer_target_scroll_frame_test.pack(fill=tk.X, expand=True, pady=5, padx=5)
        self._create_dynamic_target_widgets(self.test_target_params_frame, num_targets_var, self.test_target_entries) 

        ttk.Label(scrollable_inner_frame, text="--- Imaging Parameters ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(scrollable_inner_frame, self.test_params, "image_x_min", "Image X Min (mm):", -20)
        self.add_param(scrollable_inner_frame, self.test_params, "image_x_max", "Image X Max (mm):", 20)
        self.add_param(scrollable_inner_frame, self.test_params, "image_y_min", "Image Y Min (mm):", -40)
        self.add_param(scrollable_inner_frame, self.test_params, "image_y_max", "Image Y Max (mm):", -5)
        self.add_param(scrollable_inner_frame, self.test_params, "image_resolution_mm", "Image Resolution (mm):", 1.0) 

        self.run_testbed_button = ttk.Button(scrollable_inner_frame, text="Run Analytical Testbed", command=self.run_testbed)
        self.run_testbed_button.pack(pady=10, fill=tk.X, padx=5)
        self.testbed_progress_bar = ttk.Progressbar(scrollable_inner_frame, mode='determinate'); self.testbed_progress_bar.pack(pady=2, fill=tk.X, padx=5)
        self.testbed_status_label = ttk.Label(scrollable_inner_frame, text="Status: Idle"); self.testbed_status_label.pack(pady=2, fill=tk.X, padx=5)

        self.update_test_spacing()

        self.fig_test_plots = plt.figure(figsize=(10, 9))
        gs = self.fig_test_plots.add_gridspec(2, 2, height_ratios=[1, 1], width_ratios=[1,1]) 

        self.ax_test_scenario = self.fig_test_plots.add_subplot(gs[0, 0])
        self.ax_test_das_2d = self.fig_test_plots.add_subplot(gs[0, 1])
        self.ax_test_angular_spectrum = self.fig_test_plots.add_subplot(gs[1, :]) 
        
        self.fig_test_plots.tight_layout(pad=3.0, h_pad=5.0)
        self.canvas_test = FigureCanvasTkAgg(self.fig_test_plots, master=plot_frame); self.canvas_test.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        self.plot_test_all_empty()
        print("ULTRA-DEBUG: create_algorithm_testbed_tab completed.")

    def run_testbed(self):
        if self.testbed_thread and self.testbed_thread.is_alive():
            messagebox.showwarning("Testbed Running", "An analytical testbed run is already in progress.")
            return

        try:
            targets_enabled_state = self.test_targets_enabled_var.get() 
            layers_enabled_state = self.test_layers_enabled_var.get()

            p_check = {name: var.get() for name, var in self.test_params.items()}
            
            if p_check['test_freq'] <= 0 or p_check['test_num_rx_antennas'] <= 0 or \
               p_check['test_num_tx_antennas'] <= 0 or p_check['image_resolution_mm'] <= 0 or \
               p_check['tx_power_dbm'] < -100 or p_check['rx_gain_db'] < -50 or p_check['noise_figure_db'] < 0 or \
               p_check['bandwidth_mhz'] <= 0 or p_check['system_temp_k'] <= 0:
                messagebox.showerror("Input Error", "All numeric parameters must be valid (e.g., positive for freq/bandwidth/temp, reasonable for power/gain/NF).")
                return
            
            if layers_enabled_state and p_check['test_skin_surface_offset'] <= 0:
                messagebox.showerror("Input Error", "Skin Surface Offset must be positive if layers are enabled.")
                return

            if targets_enabled_state and p_check['test_num_targets'] <= 0:
                messagebox.showerror("Input Error", "Number of targets must be positive if enabled.")
                return
            
            if p_check['image_x_min'] >= p_check['image_x_max'] or p_check['image_y_min'] >= p_check['image_y_max']:
                messagebox.showerror("Input Error", "Image X/Y Max must be greater than Min.")
                return

        except (tk.TclError, ValueError) as e:
            messagebox.showerror("Input Error", f"Invalid numerical input provided: {e}")
            return

        self.run_testbed_button.config(state='disabled')
        self.testbed_results = None
        self.testbed_status_label.config(text="Status: Initializing Simulation...")
        
        if self.testbed_progress_bar:
            self.testbed_progress_bar['value'] = 0
            self.testbed_progress_bar['maximum'] = 100 
            self.testbed_progress_bar['mode'] = 'determinate'

        self.testbed_thread = threading.Thread(target=self._execute_testbed_thread)
        self.testbed_thread.daemon = True 
        self.testbed_thread.start()
        self.after(100, self._check_testbed_status)

    def _execute_testbed_thread(self):
        try:
            p = {name: var.get() for name, var in self.test_params.items()}
            p['layers_enabled'] = self.test_layers_enabled_var.get()
            p['targets_enabled'] = self.test_targets_enabled_var.get() 
            
            analytical_results = self._perform_analytical_sim_and_beamforming(p, self.test_layer_entries, self.test_target_entries, self.testbed_status_label, self.testbed_progress_bar, 0)
            
            self.testbed_results = analytical_results 
        except Exception as e: 
            import traceback
            self.testbed_results = {"error": f"{e}\n{traceback.format_exc()}"}

    def _check_testbed_status(self):
        if self.testbed_results:
            self.run_testbed_button.config(state='normal')
            if self.testbed_progress_bar: self.testbed_progress_bar.stop()
            if "error" in self.testbed_results: 
                messagebox.showerror("Analytical Testbed Error", str(self.testbed_results["error"]))
                self.testbed_status_label.config(text="Status: Error occurred.")
            else:
                data = self.testbed_results
                
                tx_pos_x = data.get('tx_pos_x', np.array([]))
                rx_pos_x = data.get('rx_pos_x', np.array([]))
                targets = data.get('targets', [])
                layers = data.get('layers', [])
                skin_surface_y_plot = data.get('skin_surface_y_plot', Y_ANTENNA_PLANE)

                self.plot_test_scenario(targets, layers, tx_pos_x, rx_pos_x, skin_surface_y_plot)
                self.plot_test_das_2d_image(data['image_x_grid'], data['image_y_grid'], data['das_image_2d'], targets, skin_surface_y_plot)
                self.plot_test_angular_spectrum(data['angles_deg_1d'], data['das_spectrum_1d'], data['music_spectrum_1d'], data['capon_spectrum_1d'], data['true_angle_deg_1d'])
                
                self.canvas_test.draw()
                self.testbed_status_label.config(text="Status: Test Complete")
            
            self.testbed_thread = None 
        else: 
            self.testbed_status_label.config(text="Status: Running...")
            self.after(100, self._check_testbed_status)

    def _perform_analytical_sim_and_beamforming(self, p, layer_entries, target_entries, status_label_widget, progress_bar_widget, initial_progress):
        num_tx = int(p.get('test_num_tx_antennas', p.get('v2_num_tx')))
        num_rx = int(p.get('test_num_rx_antennas', p.get('v2_num_rx')))
        d_mm = p.get('test_spacing', p.get('v2_spacing'))
        f_ghz = p.get('test_freq', p.get('v2_freq'))
        
        tx_power_dbm = p.get('tx_power_dbm', p.get('v2_tx_power_dbm'))
        rx_gain_db = p.get('rx_gain_db', p.get('v2_rx_gain_db'))
        noise_figure_db = p.get('noise_figure_db', p.get('v2_noise_figure_db'))
        bandwidth_mhz = p.get('bandwidth_mhz', p.get('v2_bandwidth_mhz'))
        system_temp_k = p.get('system_temp_k', p.get('v2_system_temp_k'))

        tx_power_W = 10**((tx_power_dbm - 30) / 10)
        rx_gain_linear = 10**(rx_gain_db / 10)
        noise_figure_linear = 10**(noise_figure_db / 10)
        bandwidth_hz = bandwidth_mhz * 1e6

        noise_power_W = k * system_temp_k * bandwidth_hz * noise_figure_linear
        noise_amplitude_per_channel = np.sqrt(noise_power_W / 2) 

        tx_pos_x = np.arange(num_tx) * d_mm - (num_tx - 1) * d_mm / 2
        rx_pos_x = np.arange(num_rx) * d_mm - (num_rx - 1) * d_mm / 2
        
        fc = f_ghz * 1e9 
        lambda_mm = (c / fc) * 1000 
        k_vacuum = 2 * np.pi / lambda_mm 
        
        layers_enabled = p.get('layers_enabled', p.get('v2_layers_enabled'))
        layers = self.get_layer_params(layer_entries) if layers_enabled else [] 
        skin_surface_offset = p.get('test_skin_surface_offset', p.get('v2_skin_surface_offset'))
        skin_surface_y_plot = Y_ANTENNA_PLANE - skin_surface_offset

        targets_enabled = p.get('targets_enabled', p.get('v2_targets_enabled'))
        targets = self.get_target_params(target_entries) if targets_enabled else [] 

        mimo_matrix_signals = np.zeros((num_tx, num_rx), dtype=np.complex128)

        status_label_widget.config(text="Status: Analytical - Generating Signals...")
        if progress_bar_widget: progress_bar_widget['value'] = initial_progress + 5

        for tx_idx in range(num_tx):
            tx_antenna_pos = np.array([tx_pos_x[tx_idx], Y_ANTENNA_PLANE])
            for rx_idx in range(num_rx):
                rx_antenna_pos = np.array([rx_pos_x[rx_idx], Y_ANTENNA_PLANE])
                
                received_signal_field = 0.0
                if targets_enabled and len(targets) > 0:
                    for target in targets:
                        target_center_pos = np.array([target['x'], target['y']])
                        
                        total_path_length_tx_target_mm, phase_tx_target, total_attenuation_tx_target_db = self._calculate_layered_path_metrics(
                            tx_antenna_pos, target_center_pos, layers, f_ghz, skin_surface_y_plot
                        )
                        
                        total_path_length_target_rx_mm, phase_target_rx, total_attenuation_target_rx_db = self._calculate_layered_path_metrics(
                            target_center_pos, rx_antenna_pos, layers, f_ghz, skin_surface_y_plot
                        )
                        
                        dist_tx_target_m = total_path_length_tx_target_mm * 1e-3
                        dist_target_rx_m = total_path_length_target_rx_mm * 1e-3
                        
                        dist_tx_target_m = max(dist_tx_target_m, 1e-6)
                        dist_target_rx_m = max(dist_target_rx_m, 1e-6)

                        fspl_tx_target_dB = 20 * np.log10(4 * np.pi * dist_tx_target_m / (lambda_mm * 1e-3)) 
                        fspl_target_rx_dB = 20 * np.log10(4 * np.pi * dist_target_rx_m / (lambda_mm * 1e-3)) 

                        total_path_loss_db_analytical = (fspl_tx_target_dB + fspl_target_rx_dB) \
                                                        + (total_attenuation_tx_target_db + total_attenuation_target_rx_db)
                        
                        Pr_dbm_target = tx_power_dbm + rx_gain_db \
                                        + 10*np.log10(target['rcs']) \
                                        - total_path_loss_db_analytical
                        
                        Pr_linear_target = 10**((Pr_dbm_target - 30) / 10)
                        
                        field_amplitude_at_rx = np.sqrt(max(0, Pr_linear_target))
                        
                        received_signal_field += field_amplitude_at_rx * np.exp(-1j * (phase_tx_target + phase_target_rx))

                noise_real = np.random.normal(0, noise_amplitude_per_channel)
                noise_imag = np.random.normal(0, noise_amplitude_per_channel)
                noise_complex = noise_real + 1j * noise_imag

                mimo_matrix_signals[tx_idx, rx_idx] = received_signal_field + noise_complex
        
        status_label_widget.config(text="Status: Analytical - Performing 2D Beamforming...")
        if progress_bar_widget: progress_bar_widget['value'] = initial_progress + 15

        image_x_min = p.get('image_x_min', p.get('v2_image_x_min'))
        image_x_max = p.get('image_x_max', p.get('v2_image_x_max'))
        image_y_min = p.get('image_y_min', p.get('v2_image_y_min'))
        image_y_max = p.get('image_y_max', p.get('v2_image_y_max'))
        image_resolution_mm = p.get('image_resolution_mm', p.get('v2_image_resolution_mm'))

        image_x_grid = np.arange(image_x_min, image_x_max + image_resolution_mm, image_resolution_mm)
        image_y_grid = np.arange(image_y_min, image_y_max + image_resolution_mm, image_resolution_mm)
        
        skin_surface_y_plot = Y_ANTENNA_PLANE - skin_surface_offset
        image_y_grid_filtered = image_y_grid[image_y_grid < skin_surface_y_plot + 1e-9] 
        
        das_image_2d = np.zeros((len(image_x_grid), len(image_y_grid_filtered)), dtype=np.complex128)

        for i, x_pixel in enumerate(image_x_grid):
            for j, y_pixel in enumerate(image_y_grid_filtered):
                pixel_pos = np.array([x_pixel, y_pixel])
                focused_sum = 0
                for tx_idx in range(num_tx):
                    for rx_idx in range(num_rx):
                        tx_antenna_pos = np.array([tx_pos_x[tx_idx], Y_ANTENNA_PLANE])
                        rx_antenna_pos = np.array([rx_pos_x[rx_idx], Y_ANTENNA_PLANE])
                        
                        path_metrics_tx_pixel = self._calculate_layered_path_metrics(
                            tx_antenna_pos, pixel_pos, layers, f_ghz, skin_surface_y_plot
                        )
                        _, phase_tx_pixel, _ = path_metrics_tx_pixel

                        path_metrics_pixel_rx = self._calculate_layered_path_metrics(
                            pixel_pos, rx_antenna_pos, layers, f_ghz, skin_surface_y_plot
                        )
                        _, phase_pixel_rx, _ = path_metrics_pixel_rx
                        
                        total_phase_correction = phase_tx_pixel + phase_pixel_rx
                        
                        focused_sum += mimo_matrix_signals[tx_idx, rx_idx] * np.exp(1j * total_phase_correction)
                das_image_2d[i, j] = focused_sum
        
        status_label_widget.config(text="Status: Analytical - Performing 1D Angular Beamforming...")
        if progress_bar_widget: progress_bar_widget['value'] = initial_progress + 25

        central_tx_idx = num_tx // 2
        signals_for_angular_spectrum = mimo_matrix_signals[central_tx_idx, :]
        
        angles = np.linspace(-np.pi/2, np.pi/2, 361) 
        das_spectrum_1d = np.zeros(len(angles)); music_spectrum_1d = np.zeros(len(angles)); capon_spectrum_1d = np.zeros(len(angles))
        
        R_1d = np.outer(signals_for_angular_spectrum, np.conj(signals_for_angular_spectrum))
        
        loading_factor_1d = 1e-9 * np.max(np.abs(R_1d)) if np.max(np.abs(R_1d)) > 1e-12 else 1e-9
        R_1d += np.eye(num_rx) * loading_factor_1d 
        
        R_1d_inv = None
        try:
            R_1d_inv = np.linalg.inv(R_1d)
        except np.linalg.LinAlgError:
            print("WARNING: R_1d is singular even after loading in analytical testbed. Capon/MUSIC skipped.")

        eigvals_1d, eigvecs_1d = np.linalg.eigh(R_1d)
        
        true_angle_deg = 0.0
        if targets_enabled and len(targets) > 0:
            first_target = targets[0]
            target_pos_for_angle = np.array([first_target['x'], first_target['y']])
            true_angle_rad = np.arctan2(target_pos_for_angle[0], abs(target_pos_for_angle[1] - Y_ANTENNA_PLANE))
            true_angle_deg = np.degrees(true_angle_rad)

        for i, angle in enumerate(angles):
            steering_vector_1d = np.exp(1j * k_vacuum * rx_pos_x * np.sin(angle)) 
            steering_vector_1d_col = steering_vector_1d.reshape(-1, 1)

            das_spectrum_1d[i] = np.abs(np.vdot(steering_vector_1d, signals_for_angular_spectrum))**2
            
            if num_rx > 1 and R_1d_inv is not None:
                noise_subspace_1d = eigvecs_1d[:, :-1]
                noise_proj_1d = steering_vector_1d_col.T.conj() @ noise_subspace_1d
                denominator_music_1d = np.linalg.norm(noise_proj_1d)**2
                music_spectrum_1d[i] = 1 / (denominator_music_1d + 1e-9)
            else:
                music_spectrum_1d[i] = 0.0

            if R_1d_inv is not None:
                denominator_capon_1d = steering_vector_1d_col.T.conj() @ R_1d_inv @ steering_vector_1d_col
                capon_spectrum_1d[i] = 1 / (np.abs(denominator_capon_1d.item()) + 1e-9) 
            else:
                capon_spectrum_1d[i] = 0.0
        
        return {
            "params": p, 
            "layers": layers,
            "targets": targets,
            "mimo_matrix_signals": mimo_matrix_signals,
            "skin_surface_y_plot": skin_surface_y_plot,
            "tx_pos_x": tx_pos_x,
            "rx_pos_x": rx_pos_x,
            "image_x_grid": image_x_grid,
            "image_y_grid": image_y_grid_filtered,
            "das_image_2d": np.abs(das_image_2d)**2,
            "angles_deg_1d": np.degrees(angles),
            "das_spectrum_1d": das_spectrum_1d / (np.max(das_spectrum_1d) + 1e-9),
            "music_spectrum_1d": music_spectrum_1d / (np.max(music_spectrum_1d) + 1e-9),
            "capon_spectrum_1d": capon_spectrum_1d / (np.max(capon_spectrum_1d) + 1e-9),
            "true_angle_deg_1d": true_angle_deg
        }


    def _calculate_layered_path_metrics(self, start_point_mm, end_point_mm, layer_params, f_ghz, skin_surface_y_plot_coord):
        total_phase_shift = 0.0
        total_attenuation_dB = 0.0
        total_path_length_mm = 0.0

        fc = f_ghz * 1e9
        omega = 2 * np.pi * fc
        
        layer_properties = [] 
        current_y_top = skin_surface_y_plot_coord
        
        layer_properties.append((Y_ANTENNA_PLANE + 1000, skin_surface_y_plot_coord, 1.0, 0.0)) 

        if layer_params:
            for layer in layer_params:
                layer_bottom_y = current_y_top - layer['thickness']
                layer_properties.append((current_y_top, layer_bottom_y, layer['permit'], layer['conductivity']))
                current_y_top = layer_bottom_y
        
        layer_properties.append((current_y_top, current_y_top - 1000, 1.0, 0.0)) 

        start_p = np.array(start_point_mm)
        end_p = np.array(end_point_mm)

        all_y_coords = sorted(list(set([start_p[1], end_p[1]] + [b[0] for b in layer_properties] + [b[1] for b in layer_properties])))
        
        min_y_ray, max_y_ray = min(start_p[1], end_p[1]), max(start_p[1], end_p[1])
        segment_y_coords = [y for y in all_y_coords if min_y_ray - 1e-9 <= y <= max_y_ray + 1e-9]
        segment_y_coords = sorted(list(set(segment_y_coords)))

        if len(segment_y_coords) < 2:
            mid_y_for_single_segment = (start_p[1] + end_p[1]) / 2.0
            if np.isclose(start_p[1], end_p[1]) and np.isclose(start_p[0], end_p[0]): 
                mid_y_for_single_segment += 1e-6 
            elif np.isclose(start_p[1], end_p[1]): 
                pass 

            medium_permit_real = 1.0
            medium_conductivity_s_m = 0.0
            for top_y, bottom_y, permit_r, cond_s_m in layer_properties:
                if mid_y_for_single_segment <= top_y + 1e-9 and mid_y_for_single_segment >= bottom_y - 1e-9:
                    medium_permit_real = permit_r
                    medium_conductivity_s_m = cond_s_m
                    break
            
            length = np.linalg.norm(end_p - start_p)
            total_path_length_mm = length
            
            epsilon_complex_val = (medium_permit_real * epsilon_0) - 1j * (medium_conductivity_s_m / omega)
            gamma = 1j * omega * np.sqrt(mu_0 * epsilon_complex_val)
            
            total_phase_shift = length * (np.imag(gamma) / 1000.0) 
            total_attenuation_dB = length * (np.real(gamma) * (20 * np.log10(np.exp(1))) / 1000.0) 
            
            return total_path_length_mm, total_phase_shift, total_attenuation_dB


        for i in range(len(segment_y_coords) - 1):
            seg_y1 = segment_y_coords[i]
            seg_y2 = segment_y_coords[i+1]
            
            if not np.isclose(start_p[1], end_p[1]): 
                seg_x1 = start_p[0] + (end_p[0] - start_p[0]) * (seg_y1 - start_p[1]) / (end_p[1] - start_p[1])
                seg_x2 = start_p[0] + (end_p[0] - start_p[0]) * (seg_y2 - start_p[1]) / (end_p[1] - start_p[1])
            else: 
                seg_x1 = start_p[0]
                seg_x2 = end_p[0]

            seg_start_p = np.array([seg_x1, seg_y1])
            seg_end_p = np.array([seg_x2, seg_y2])
            seg_length = np.linalg.norm(seg_end_p - seg_start_p) 
            
            mid_y_seg = (seg_y1 + seg_y2) / 2.0
            
            medium_permit_real = 1.0
            medium_conductivity_s_m = 0.0
            for top_y, bottom_y, permit_r, cond_s_m in layer_properties:
                if mid_y_seg <= top_y + 1e-9 and mid_y_seg >= bottom_y - 1e-9:
                    medium_permit_real = permit_r
                    medium_conductivity_s_m = cond_s_m
                    break
            
            epsilon_complex_val = (medium_permit_real * epsilon_0) - 1j * (medium_conductivity_s_m / omega)
            gamma = 1j * omega * np.sqrt(mu_0 * epsilon_complex_val)
            
            total_path_length_mm += seg_length
            total_phase_shift += seg_length * (np.imag(gamma) / 1000.0) 
            total_attenuation_dB += seg_length * (np.real(gamma) * (20 * np.log10(np.exp(1))) / 1000.0) 
            
        return total_path_length_mm, total_phase_shift, total_attenuation_dB


    def calculate_path_phase(self, start_pos, end_pos, layers, f_ghz): 
        _, phase_shift, _ = self._calculate_layered_path_metrics(start_pos, end_pos, layers, f_ghz, Y_ANTENNA_PLANE)
        return np.exp(-1j * phase_shift)
        

    def add_param(self, parent, param_dict, name, label_text, default_value, return_var=False, return_var_and_widget=False, readonly=False, layout_manager='pack', **kwargs):
        param_container_frame = ttk.Frame(parent)
        
        param_container_frame.grid_columnconfigure(1, weight=1)

        ttk.Label(param_container_frame, text=label_text, width=22).grid(row=0, column=0, sticky=tk.W, padx=5, pady=2)
        var = tk.DoubleVar(value=default_value)
        param_dict[name] = var 
        entry = ttk.Entry(param_container_frame, textvariable=var)
        if readonly:
            entry.config(state='readonly')
        entry.grid(row=0, column=1, sticky=tk.EW, padx=5, pady=2)

        if layout_manager == 'pack':
            param_container_frame.pack(fill=tk.X, pady=2, padx=5, **kwargs)
        elif layout_manager == 'grid':
            param_container_frame.grid(**kwargs) 
        else:
            raise ValueError(f"Unsupported layout_manager: {layout_manager}. Choose 'pack' or 'grid'.")

        if return_var_and_widget: return {'var': var, 'widget': entry}
        return var if return_var else entry

    def _create_dynamic_layer_widgets(self, parent_frame_for_widgets, num_layers_var, entries_list, default_permit=38, default_conductivity=0.1): 
        layer_params_actual_frame = parent_frame_for_widgets 

        def update_widgets(*args):
            for widget in layer_params_actual_frame.winfo_children(): widget.destroy()
            entries_list.clear()
            try: num = int(num_layers_var.get())
            except (ValueError, tk.TclError): num = 0
            
            defaults = [
                {'thickness': 1.5, 'permit': 38, 'name': 'Epidermis', 'conductivity': 0.1}, 
                {'thickness': 2.5, 'permit': 45, 'name': 'Dermis', 'conductivity': 0.08}
            ]
            
            for i in range(num):
                default_val = defaults[i] if i < len(defaults) else {'thickness': 2.0, 'permit': default_permit, 'name': f'Layer {i+1}', 'conductivity': default_conductivity}
                
                frame = ttk.LabelFrame(layer_params_actual_frame, text=default_val['name']); frame.pack(fill=tk.X, pady=3, padx=5)
                frame.grid_columnconfigure(1, weight=1)

                t_var, p_var, c_var = tk.DoubleVar(value=default_val['thickness']), tk.DoubleVar(value=default_val['permit']), tk.DoubleVar(value=default_val['conductivity'])
                
                ttk.Label(frame, text="Thickness (mm):").grid(row=0, column=0, sticky=tk.W, padx=5); ttk.Entry(frame, textvariable=t_var).grid(row=0, column=1, sticky=tk.EW, padx=5)
                ttk.Label(frame, text="Permittivity (εᵣ):").grid(row=1, column=0, sticky=tk.W, padx=5); ttk.Entry(frame, textvariable=p_var).grid(row=1, column=1, sticky=tk.EW, padx=5)
                ttk.Label(frame, text="Conductivity (S/m):").grid(row=2, column=0, sticky=tk.W, padx=5); ttk.Entry(frame, textvariable=c_var).grid(row=2, column=1, sticky=tk.EW, padx=5)
                
                layer_entry_dict = {'thickness': t_var, 'permit': p_var, 'conductivity': c_var}

                entries_list.append(layer_entry_dict)
            
            if hasattr(layer_params_actual_frame.master, 'yview_scroll'): 
                layer_params_actual_frame.update_idletasks()
                layer_params_actual_frame.master.config(scrollregion=layer_params_actual_frame.master.bbox("all"))

        num_layers_var.trace("w", update_widgets); update_widgets()
        return layer_params_actual_frame 

    def _create_dynamic_target_widgets(self, parent_frame_for_widgets, num_targets_var, entries_list, target_permit_default=60, target_conductivity_default=0.1): 
        target_params_actual_frame = parent_frame_for_widgets 
        def update_widgets(*args):
            for widget in target_params_actual_frame.winfo_children(): widget.destroy()
            entries_list.clear()
            try: num = int(num_targets_var.get())
            except (ValueError, tk.TclError): num = 0
            
            defaults = [ 
                {'x': 0.0, 'y': -4.0, 'radius': 0.75, 'rcs': 0.001, 'permit': 60, 'conductivity': 0.1},
                {'x': -5.0, 'y': -20.0, 'radius': 1.0, 'rcs': 0.005, 'permit': 55, 'conductivity': 0.08}
            ]

            for i in range(num):
                default = defaults[i] if i < len(defaults) else {'x': 0.0, 'y': -25.0, 'radius': 0.75, 'rcs': 1.0, 'permit': target_permit_default, 'conductivity': target_conductivity_default}
                
                frame = ttk.LabelFrame(target_params_actual_frame, text=f"Target {i+1}"); frame.pack(fill=tk.X, pady=3, padx=5)
                frame.grid_columnconfigure(1, weight=1)

                x_var, y_var = tk.DoubleVar(value=default['x']), tk.DoubleVar(value=default['y'])
                r_var, rcs_var = tk.DoubleVar(value=default['radius']), tk.DoubleVar(value=default['rcs'])
                p_var = tk.DoubleVar(value=default['permit']) 
                c_var = tk.DoubleVar(value=default['conductivity']) 

                ttk.Label(frame, text="X Pos (mm):").grid(row=0, column=0, sticky=tk.W, padx=5); ttk.Entry(frame, textvariable=x_var).grid(row=0, column=1, sticky=tk.EW, padx=5)
                ttk.Label(frame, text="Y Pos (mm):").grid(row=1, column=0, sticky=tk.W, padx=5); ttk.Entry(frame, textvariable=y_var).grid(row=1, column=1, sticky=tk.EW, padx=5)
                ttk.Label(frame, text="Radius (mm):").grid(row=2, column=0, sticky=tk.W, padx=5); ttk.Entry(frame, textvariable=r_var).grid(row=2, column=1, sticky=tk.EW, padx=5)
                ttk.Label(frame, text="Reflectivity (RCS, m²):").grid(row=3, column=0, sticky=tk.W, padx=5); ttk.Entry(frame, textvariable=rcs_var).grid(row=3, column=1, sticky=tk.EW, padx=5)
                ttk.Label(frame, text="Permittivity (εᵣ):").grid(row=4, column=0, sticky=tk.W, padx=5); ttk.Entry(frame, textvariable=p_var).grid(row=4, column=1, sticky=tk.EW, padx=5)
                ttk.Label(frame, text="Conductivity (S/m):").grid(row=5, column=0, sticky=tk.W, padx=5); ttk.Entry(frame, textvariable=c_var).grid(row=5, column=1, sticky=tk.EW, padx=5)
                
                entries_list.append({'x': x_var, 'y': y_var, 'radius': r_var, 'rcs': rcs_var, 'permit': p_var, 'conductivity': c_var})

            if hasattr(target_params_actual_frame.master, 'yview_scroll'):
                target_params_actual_frame.update_idletasks()
                target_params_actual_frame.master.config(scrollregion=target_params_actual_frame.master.bbox("all"))

        num_targets_var.trace("w", update_widgets); update_widgets()
        return target_params_actual_frame 

    def toggle_spacing_entry(self):
        self.spacing_entry.config(state='disabled' if self.auto_spacing_var.get() else 'normal')
    
    def get_layer_params(self, entries_list): 
        return [{'thickness': e['thickness'].get(), 'permit': e['permit'].get(), 'conductivity': e['conductivity'].get()} for e in entries_list]
    
    def get_target_params(self, entries_list): 
        return [{'x': e['x'].get(), 'y': e['y'].get(), 'radius': e['radius'].get(), 'rcs': e['rcs'].get(), 'permit': e['permit'].get(), 'conductivity': e['conductivity'].get()} for e in entries_list] 
    
    def get_fw_target_params(self, entries_list): 
        return [{'x': e['x'].get(), 'y': e['y'].get(), 'radius': e['radius'].get(), 'permit': e['permit'].get(), 'conductivity': e['conductivity'].get()} for e in entries_list]

    def _toggle_dynamic_widget_frame(self, enable_var, num_entry_widget, params_frame_inner):
        state = 'normal' if enable_var.get() else 'disabled'
        
        if num_entry_widget:
            num_entry_widget.config(state=state)
        
        for child_frame in params_frame_inner.winfo_children():
            if isinstance(child_frame, ttk.LabelFrame): 
                for widget_in_child in child_frame.winfo_children():
                    if hasattr(widget_in_child, 'config') and 'state' in widget_in_child.config():
                        widget_in_child.config(state=state)
            elif hasattr(child_frame, 'config') and 'state' in child_frame.config(): 
                 child_frame.config(state=state)

    def toggle_test_layer_entries(self): 
        state = 'normal' if self.test_layers_enabled_var.get() else 'disabled'
        if self.test_skin_offset_entry_widget:
            self.test_skin_offset_entry_widget.config(state=state)
        self._toggle_dynamic_widget_frame(self.test_layers_enabled_var, self.test_num_layers_entry_widget, self.test_layer_params_frame)

    def toggle_test_target_entries(self): 
        self._toggle_dynamic_widget_frame(self.test_targets_enabled_var, self.test_num_targets_entry_widget, self.test_target_params_frame)

    def toggle_test_spacing_entry(self): 
        self.test_spacing_entry.config(state='disabled' if self.test_auto_spacing_var.get() else 'normal')
        self.update_test_spacing()

    def update_test_spacing(self, *args): 
        if self.test_auto_spacing_var.get():
            try:
                freq_ghz = self.test_params['test_freq'].get()
                if freq_ghz > 0:
                    lambda_mm = (c / (freq_ghz * 1e9)) * 1000
                    self.test_params['test_spacing'].set(round(lambda_mm / 2, 4))
            except (tk.TclError, KeyError): pass

    def toggle_fw_auto_spacing_entry(self):
        self.fw_spacing_entry.config(state='disabled' if self.fw_auto_spacing_var.get() else 'normal')
        self.update_fw_spacing()

    def update_fw_spacing(self, *args):
        if self.fw_auto_spacing_var.get():
            try:
                freq_ghz = self.fw_test_params['fw_freq_max'].get()
                if freq_ghz > 0:
                    lambda_mm = (c / (freq_ghz * 1e9)) * 1000
                    self.fw_test_params['fw_antenna_spacing'].set(round(lambda_mm / 2, 4))
            except (tk.TclError, KeyError): pass

    def toggle_fw_layer_entries(self):
        state = 'normal' if self.fw_layers_enabled_var.get() else 'disabled'
        if self.fw_skin_offset_entry_widget:
            self.fw_skin_offset_entry_widget.config(state=state)
        self._toggle_dynamic_widget_frame(self.fw_layers_enabled_var, self.fw_num_layers_entry_widget, self.fw_layer_params_frame)

    def toggle_fw_target_entries(self):
        self._toggle_dynamic_widget_frame(self.fw_targets_enabled_var, self.fw_num_targets_entry_widget, self.fw_target_params_frame)

    def create_full_wave_em_testbed_tab(self):
        main_frame = ttk.Frame(self.tab3, padding="5", style='App.TFrame')
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        control_outer_frame, scrollable_inner_frame = self._create_scrollable_param_frame(main_frame, text_label="FDTD Parameters")
        control_outer_frame.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)
        
        plot_frame = ttk.Frame(main_frame, padding="5", style='App.TFrame')
        plot_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        self.fw_test_params = {}

        ttk.Label(scrollable_inner_frame, text="--- MIMO Antenna Array ---", style='Header.TLabel').pack(pady=5, anchor='w')
        self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_num_tx", "Num Transmitters:", 4)
        self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_num_rx", "Num Receivers:", 16)
        self.fw_freq_var = self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_freq_max", "Center Frequency (GHz):", 77, return_var=True)
        spacing_frame = ttk.Frame(scrollable_inner_frame); spacing_frame.pack(fill=tk.X, pady=2)
        ttk.Checkbutton(spacing_frame, text="Auto-calculate spacing (λ/2)", variable=self.fw_auto_spacing_var, command=self.toggle_fw_auto_spacing_entry).pack(anchor='w', padx=5)
        returned_widgets = self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_antenna_spacing", "Antenna Spacing (mm):", 1.95, return_var_and_widget=True)
        self.fw_spacing_entry = returned_widgets['widget']
        self.fw_freq_var.trace("w", self.update_fw_spacing)
        
        ttk.Label(scrollable_inner_frame, text="--- FDTD Simulation Control ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_resolution", "Sim Resolution (pixels/mm):", 15) 
        self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_pml_thickness", "PML Thickness (mm):", 2.0)

        ttk.Label(scrollable_inner_frame, text="--- Medium Properties (FDTD) ---", style='Header.TLabel').pack(pady=10, anchor='w')
        ttk.Checkbutton(scrollable_inner_frame, text="Enable Skin Layers", variable=self.fw_layers_enabled_var, command=self.toggle_fw_layer_entries).pack(anchor='w', padx=5)
        fw_skin_offset_info = self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_skin_surface_offset", "Skin Surface Offset (mm):", 2.5, return_var_and_widget=True) 
        self.fw_skin_offset_entry_widget = fw_skin_offset_info['widget']
        
        self.fw_layer_entries = []
        fw_num_layers_info = self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_num_layers", "Num Layers:", 2, return_var_and_widget=True)
        fw_num_layers_var = fw_num_layers_info['var']
        self.fw_num_layers_entry_widget = fw_num_layers_info['widget']
        
        outer_layer_scroll_frame_fw, self.fw_layer_params_frame = self._create_scrollable_param_frame(scrollable_inner_frame, text_label="Skin Layers Properties", min_canvas_height=200)
        outer_layer_scroll_frame_fw.pack(fill=tk.X, expand=True, pady=5, padx=5)
        self._create_dynamic_layer_widgets(self.fw_layer_params_frame, fw_num_layers_var, self.fw_layer_entries) 

        ttk.Label(scrollable_inner_frame, text="--- Multiple Targets (FDTD) ---", style='Header.TLabel').pack(pady=10, anchor='w')
        ttk.Checkbutton(scrollable_inner_frame, text="Enable Multiple Targets", variable=self.fw_targets_enabled_var, command=self.toggle_fw_target_entries).pack(anchor='w', padx=5)

        self.fw_target_entries = []
        fw_num_targets_info = self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_num_targets", "Number of Targets:", 1, return_var_and_widget=True)
        fw_num_targets_var = fw_num_targets_info['var']
        self.fw_num_targets_entry_widget = fw_num_targets_info['widget']
        
        outer_target_scroll_frame_fw, self.fw_target_params_frame = self._create_scrollable_param_frame(scrollable_inner_frame, text_label="Target Properties", min_canvas_height=200)
        outer_target_scroll_frame_fw.pack(fill=tk.X, expand=True, pady=5, padx=5)
        self._create_dynamic_target_widgets(self.fw_target_params_frame, fw_num_targets_var, self.fw_target_entries) 

        ttk.Label(scrollable_inner_frame, text="--- Imaging Parameters ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_image_x_min", "Image X Min (mm):", -20)
        self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_image_x_max", "Image X Max (mm):", 20)
        self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_image_y_min", "Image Y Min (mm):", -40)
        self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_image_y_max", "Image Y Max (mm):", -5)
        self.add_param(scrollable_inner_frame, self.fw_test_params, "fw_image_resolution_mm", "Image Resolution (mm):", 1.0) 

        self.run_full_wave_testbed_button = ttk.Button(scrollable_inner_frame, text="Run FDTD Testbed Simulation", command=self.run_full_wave_testbed)
        self.run_full_wave_testbed_button.pack(pady=10, fill=tk.X, padx=5)
        self.fw_testbed_progress_bar = ttk.Progressbar(scrollable_inner_frame, mode='determinate'); self.fw_testbed_progress_bar.pack(pady=2, fill=tk.X, padx=5)
        self.fw_testbed_status_label = ttk.Label(scrollable_inner_frame, text="Status: Idle"); self.fw_testbed_status_label.pack(pady=2, fill=tk.X, padx=5)

        self.update_fw_spacing() 

        self.fig_fw_test_plots = plt.figure(figsize=(10, 9))
        gs_fw = self.fig_fw_test_plots.add_gridspec(2, 2, height_ratios=[1, 1])

        self.ax_fw_scenario = self.fig_fw_test_plots.add_subplot(gs_fw[0, 0])
        self.ax_fw_das_2d = self.fig_fw_test_plots.add_subplot(gs_fw[0, 1])
        self.ax_fw_angular_spectrum = self.fig_fw_test_plots.add_subplot(gs_fw[1, :])

        self.fig_fw_test_plots.tight_layout(pad=3.0, h_pad=5.0)
        self.canvas_fw_test = FigureCanvasTkAgg(self.fig_fw_test_plots, master=plot_frame); self.canvas_fw_test.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        self.plot_fw_all_empty()

    def run_full_wave_testbed(self):
        if self.full_wave_testbed_thread and self.full_wave_testbed_thread.is_alive():
            messagebox.showwarning("FDTD Testbed Running", "An FDTD simulation is already in progress.")
            return

        try:
            p_check = {name: var.get() for name, var in self.fw_test_params.items()}
            
            if p_check['fw_freq_max'] <= 0 or p_check['fw_num_rx'] <= 0 or \
               p_check['fw_num_tx'] <= 0 or p_check['fw_resolution'] <= 0 or p_check['fw_pml_thickness'] <= 0 or \
               p_check['fw_image_resolution_mm'] <= 0:
                messagebox.showerror("Input Error", "All numeric FDTD/Imaging parameters must be positive.")
                return

            if self.fw_layers_enabled_var.get() and p_check['fw_skin_surface_offset'] <= 0:
                messagebox.showerror("Input Error", "Skin Surface Offset must be positive if layers are enabled.")
                return

            if self.fw_targets_enabled_var.get() and p_check['fw_num_targets'] <= 0:
                messagebox.showerror("Input Error", "Number of targets must be positive if enabled in FDTD.")
                return
            
            if p_check['fw_image_x_min'] >= p_check['fw_image_x_max'] or p_check['fw_image_y_min'] >= p_check['fw_image_y_max']:
                messagebox.showerror("Input Error", "Image X/Y Max must be greater than Min.")
                return

        except (tk.TclError, ValueError) as e:
            messagebox.showerror("Input Error", f"Invalid numerical input provided: {e}")
            return

        self.run_full_wave_testbed_button.config(state='disabled')
        self.full_wave_testbed_results = None
        self.fw_testbed_status_label.config(text="Status: Initializing FDTD...")
        
        if self.fw_testbed_progress_bar:
            self.fw_testbed_progress_bar['value'] = 0
            self.fw_testbed_progress_bar['maximum'] = 100 
            self.fw_testbed_progress_bar['mode'] = 'determinate'

        self.full_wave_testbed_thread = threading.Thread(target=self._execute_full_wave_testbed_thread)
        self.full_wave_testbed_thread.daemon = True 
        self.full_wave_testbed_thread.start()
        self.after(100, self._check_full_wave_testbed_status)

    def _execute_full_wave_testbed_thread(self):
        try:
            p = {name: var.get() for name, var in self.fw_test_params.items()}
            p['fw_layers_enabled'] = self.fw_layers_enabled_var.get()
            p['fw_targets_enabled'] = self.fw_targets_enabled_var.get() 

            fdtd_results = self._perform_fdtd_sim_and_beamforming(
                p, self.fw_layer_entries, self.fw_target_entries, self.fw_testbed_status_label, self.fw_testbed_progress_bar, 0
            )
            
            self.full_wave_testbed_results = fdtd_results
        except Exception as e: 
            import traceback
            self.full_wave_testbed_results = {"error": f"{e}\n{traceback.format_exc()}"}

    def _perform_fdtd_sim_and_beamforming(self, p, layer_entries, target_entries, status_label_widget, progress_bar_widget, initial_progress):
        num_tx = int(p.get('fw_num_tx', p.get('v2_num_tx')))
        num_rx = int(p.get('fw_num_rx', p.get('v2_num_rx')))
        d_mm = p.get('fw_antenna_spacing', p.get('v2_spacing'))
        f_ghz = p.get('fw_freq_max', p.get('v2_freq'))
        resolution = int(p.get('fw_resolution', p.get('v2_resolution')))
        pml_thickness = p.get('fw_pml_thickness', p.get('v2_pml_thickness'))
        skin_surface_offset = p.get('fw_skin_surface_offset', p.get('v2_skin_surface_offset'))
        
        tx_pos_x = np.arange(num_tx) * d_mm - (num_tx - 1) * d_mm / 2
        rx_pos_x = np.arange(num_rx) * d_mm - (num_rx - 1) * d_mm / 2
        
        fc = f_ghz * 1e9 
        lambda_mm = (c / fc) * 1000 
        k_vacuum = 2 * np.pi / lambda_mm 
        
        layers_enabled = p.get('fw_layers_enabled', p.get('v2_layers_enabled'))
        layers = self.get_layer_params(layer_entries) if layers_enabled else [] 

        targets_enabled = p.get('fw_targets_enabled', p.get('v2_targets_enabled'))
        targets = self.get_fw_target_params(target_entries) if targets_enabled else [] 

        total_skin_thickness = sum(layer['thickness'] for layer in layers)
        max_target_y_gui = min([t['y'] - t['radius'] for t in targets]) if targets else Y_ANTENNA_PLANE 
        
        max_y_depth_from_antenna_plane_mm = abs(max_target_y_gui) + skin_surface_offset + total_skin_thickness
        
        array_max_x_span_mm = (max(num_tx, num_rx) - 1) * d_mm
        max_dist_one_way_mm = np.sqrt((array_max_x_span_mm / 2)**2 + (max_y_depth_from_antenna_plane_mm + 5)**2) 
        
        max_permit_in_domain = max([l['permit'] for l in layers] + [t['permit'] for t in targets] + [1.0])
        slowest_speed_m_s = c / np.sqrt(max_permit_in_domain)
        min_physical_travel_time_s = (2 * max_dist_one_way_mm * 1e-3) / slowest_speed_m_s 
        sim_time_s = min_physical_travel_time_s * 1.5 
        run_until_meep = sim_time_s * (c / MEEP_UNIT_LENGTH_M) 

        domain_x = (num_rx + 8) * d_mm 
        domain_y = (pml_thickness * 2) + skin_surface_offset + total_skin_thickness + abs(max_target_y_gui) + 10 
        
        cell_size_meep = mp.Vector3(domain_x, domain_y, 0)
        pml_layers_meep = [mp.PML(thickness=pml_thickness)]
        
        fcen_meep = f_ghz * 1e9 * MEEP_UNIT_LENGTH_M / c
        df_meep = (f_ghz * 0.4 * 1e9) * MEEP_UNIT_LENGTH_M / c

        meep_y_antenna_plane = (domain_y / 2) - pml_thickness 

        mimo_matrix_signals_fw = np.zeros((num_tx, num_rx), dtype=np.complex128)

        total_fdtd_steps = num_tx * 2 if targets_enabled and len(targets)>0 else num_tx
        progress_per_fdtd = (30 / total_fdtd_steps) 
        current_overall_progress = initial_progress

        status_label_widget.config(text="Status: FDTD - Running Simulations...")
        
        for tx_idx in range(num_tx):
            geometry_healthy_fdtd = []
            current_y_top_meep = meep_y_antenna_plane - skin_surface_offset 
            for layer in layers:
                layer_center_y = current_y_top_meep - layer['thickness'] / 2
                geometry_healthy_fdtd.append(mp.Block(center=mp.Vector3(0, layer_center_y, 0),
                                                    size=mp.Vector3(cell_size_meep.x, layer['thickness'], mp.inf),
                                                    material=mp.Medium(epsilon=layer['permit'], 
                                                                       D_conductivity=layer['conductivity'] / (2 * np.pi * fc * epsilon_0)))) 
                current_y_top_meep -= layer['thickness']
            
            geometry_unhealthy_fdtd = list(geometry_healthy_fdtd)
            if targets_enabled and len(targets) > 0:
                for target in targets:
                    target_y_meep = meep_y_antenna_plane + target['y'] 
                    target_center_meep = mp.Vector3(target['x'], target_y_meep, 0)
                    geometry_unhealthy_fdtd.append(mp.Cylinder(center=target_center_meep, radius=target['radius'], 
                                                                height=mp.inf, 
                                                                material=mp.Medium(epsilon=target['permit'], 
                                                                                   D_conductivity=target['conductivity'] / (2 * np.pi * fc * epsilon_0))))

            tx_pos_meep = mp.Vector3(tx_pos_x[tx_idx], meep_y_antenna_plane)
            sources_meep = [mp.Source(src=mp.GaussianSource(frequency=fcen_meep, fwidth=df_meep),
                                     component=mp.Ez, center=tx_pos_meep)]
            
            status_label_widget.config(text=f"Status: FDTD for Tx {tx_idx+1}/{num_tx} (Healthy)...")
            signals_healthy_fdtd, _, _ = self._run_single_fdtd_sim_for_testbed(
                cell_size_meep, pml_layers_meep, geometry_healthy_fdtd, sources_meep, resolution,
                run_until_meep, fcen_meep, df_meep, num_rx, meep_y_antenna_plane, rx_pos_x
            )
            current_overall_progress += progress_per_fdtd / 2
            if progress_bar_widget: progress_bar_widget['value'] = current_overall_progress

            if targets_enabled and len(targets) > 0:
                status_label_widget.config(text=f"Status: FDTD for Tx {tx_idx+1}/{num_tx} (Unhealthy)...")
                signals_unhealthy_fdtd, _, _ = self._run_single_fdtd_sim_for_testbed(
                    cell_size_meep, pml_layers_meep, geometry_unhealthy_fdtd, sources_meep, resolution,
                    run_until_meep, fcen_meep, df_meep, num_rx, meep_y_antenna_plane, rx_pos_x
                )
                current_overall_progress += progress_per_fdtd / 2
                mimo_matrix_signals_fw[tx_idx, :] = signals_unhealthy_fdtd - signals_healthy_fdtd
            else:
                mimo_matrix_signals_fw[tx_idx, :] = np.zeros(num_rx, dtype=np.complex128)
                current_overall_progress += progress_per_fdtd / 2

            if progress_bar_widget: progress_bar_widget['value'] = current_overall_progress

        status_label_widget.config(text="Status: FDTD - Performing 2D Beamforming...")
        if progress_bar_widget: progress_bar_widget['value'] = initial_progress + 35

        image_x_min = p.get('fw_image_x_min', p.get('v2_image_x_min'))
        image_x_max = p.get('fw_image_x_max', p.get('v2_image_x_max'))
        image_y_min = p.get('fw_image_y_min', p.get('v2_image_y_min'))
        image_y_max = p.get('fw_image_y_max', p.get('v2_image_y_max'))
        image_resolution_mm = p.get('fw_image_resolution_mm', p.get('v2_image_resolution_mm'))

        image_x_grid = np.arange(image_x_min, image_x_max + image_resolution_mm, image_resolution_mm)
        image_y_grid = np.arange(image_y_min, image_y_max + image_resolution_mm, image_resolution_mm)
        
        skin_surface_y_plot = Y_ANTENNA_PLANE - skin_surface_offset
        image_y_grid_filtered = image_y_grid[image_y_grid < skin_surface_y_plot + 1e-9] 
        
        das_image_2d_fw = self._saft_das_beamforming_for_fw_testbed(
            mimo_matrix_signals_fw, p, image_x_grid, image_y_grid_filtered, layers, skin_surface_y_plot,
            num_tx, num_rx, d_mm, f_ghz, tx_pos_x, rx_pos_x
        )
        
        status_label_widget.config(text="Status: FDTD - Performing 1D Angular Beamforming...")
        if progress_bar_widget: progress_bar_widget['value'] = initial_progress + 45

        central_tx_idx = num_tx // 2
        signals_for_angular_spectrum_fw = mimo_matrix_signals_fw[central_tx_idx, :]
        
        angles = np.linspace(-np.pi/2, np.pi/2, 361) 
        das_spectrum_1d_fw = np.zeros(len(angles)); music_spectrum_1d_fw = np.zeros(len(angles)); capon_spectrum_1d_fw = np.zeros(len(angles))
        
        R_1d_fw = np.outer(signals_for_angular_spectrum_fw, np.conj(signals_for_angular_spectrum_fw))
        
        loading_factor_fw = 1e-9 * np.max(np.abs(R_1d_fw)) if np.max(np.abs(R_1d_fw)) > 1e-12 else 1e-9
        R_1d_fw += np.eye(num_rx) * loading_factor_fw 
        
        R_1d_inv_fw = None
        try:
            R_1d_inv_fw = np.linalg.inv(R_1d_fw)
        except np.linalg.LinAlgError:
            print("WARNING: R_1d_fw is singular even after loading in FDTD testbed. Capon/MUSIC skipped.")

        eigvals_1d_fw, eigvecs_1d_fw = np.linalg.eigh(R_1d_fw)
        
        true_angle_deg_fw = 0.0
        if targets_enabled and len(targets) > 0:
            first_target = targets[0]
            target_pos_for_angle = np.array([first_target['x'], first_target['y']])
            true_angle_rad = np.arctan2(target_pos_for_angle[0], abs(target_pos_for_angle[1] - Y_ANTENNA_PLANE))
            true_angle_deg_fw = np.degrees(true_angle_rad)

        for i, angle in enumerate(angles):
            steering_vector_1d_fw = np.exp(1j * k_vacuum * rx_pos_x * np.sin(angle)) 
            steering_vector_1d_fw_col = steering_vector_1d_fw.reshape(-1, 1)

            das_spectrum_1d_fw[i] = np.abs(np.vdot(steering_vector_1d_fw, signals_for_angular_spectrum_fw))**2
            
            if num_rx > 1 and R_1d_inv_fw is not None:
                noise_subspace_1d_fw = eigvecs_1d_fw[:, :-1]
                noise_proj_1d_fw = steering_vector_1d_fw_col.T.conj() @ noise_subspace_1d_fw
                denominator_music_1d_fw = np.linalg.norm(noise_proj_1d_fw)**2
                music_spectrum_1d_fw[i] = 1 / (denominator_music_1d_fw + 1e-9)
            else:
                music_spectrum_1d_fw[i] = 0.0

            if R_1d_inv_fw is not None:
                denominator_capon_1d_fw = steering_vector_1d_fw_col.T.conj() @ R_1d_inv_fw @ steering_vector_1d_fw_col
                capon_spectrum_1d_fw[i] = 1 / (np.abs(denominator_capon_1d_fw.item()) + 1e-9)
            else:
                capon_spectrum_1d_fw[i] = 0.0
        
        return {
            "params": p, 
            "layers": layers,
            "targets": targets,
            "mimo_matrix_signals": mimo_matrix_signals_fw,
            "skin_surface_y_plot": skin_surface_y_plot,
            "tx_pos_x": tx_pos_x,
            "rx_pos_x": rx_pos_x,
            "image_x_grid": image_x_grid,
            "image_y_grid": image_y_grid_filtered,
            "das_image_2d": np.abs(das_image_2d_fw)**2, 
            "angles_deg_1d": np.degrees(angles),
            "das_spectrum_1d": das_spectrum_1d_fw / (np.max(das_spectrum_1d_fw) + 1e-9),
            "music_spectrum_1d": music_spectrum_1d_fw / (np.max(music_spectrum_1d_fw) + 1e-9),
            "capon_spectrum_1d": capon_spectrum_1d_fw / (np.max(capon_spectrum_1d_fw) + 1e-9),
            "true_angle_deg_1d": true_angle_deg_fw
        }


    def _run_single_fdtd_sim_for_testbed(self, cell_size, pml_layers, geometry, sources, resolution, run_until_time, fcen_meep, df_meep, num_rx, meep_y_antenna_plane, rx_pos_x):
        sim = mp.Simulation(cell_size=cell_size, boundary_layers=pml_layers, geometry=geometry, sources=sources, resolution=resolution)
        
        rx_monitors = [sim.add_dft_fields([mp.Ez], fcen_meep, df_meep, 1, 
                                           where=mp.Volume(center=mp.Vector3(rx_pos_x[i], meep_y_antenna_plane, 0),
                                                           size=mp.Vector3(0,0,0))) 
                       for i in range(num_rx)]
        
        sim.run(until=run_until_time)
        
        mimo_signals = np.array([sim.get_dft_array(m, mp.Ez, 0) for m in rx_monitors]).flatten() 
        
        sim.reset_meep()
        return mimo_signals, None, None 


    def _check_full_wave_testbed_status(self):
        if self.full_wave_testbed_results:
            self.run_full_wave_testbed_button.config(state='normal')
            if self.fw_testbed_progress_bar: self.fw_testbed_progress_bar.stop()
            if "error" in self.full_wave_testbed_results: 
                messagebox.showerror("FDTD Testbed Error", str(self.full_wave_testbed_results["error"]))
                self.fw_testbed_status_label.config(text="Status: Error occurred.")
            else:
                data = self.full_wave_testbed_results
                
                fw_tx_pos_x = data.get('tx_pos_x', np.array([]))
                fw_rx_pos_x = data.get('rx_pos_x', np.array([]))
                fw_targets = data.get('targets', [])
                fw_layers = data.get('layers', [])
                fw_skin_surface_y_plot = data.get('skin_surface_y_plot', Y_ANTENNA_PLANE)

                self.plot_fw_scenario(fw_targets, fw_layers, fw_tx_pos_x, fw_rx_pos_x, fw_skin_surface_y_plot)
                self.plot_fw_das_2d_image(data['image_x_grid'], data['image_y_grid'], data['das_image_2d'], fw_targets, fw_skin_surface_y_plot)
                self.plot_fw_angular_spectrum(data['angles_deg_1d'], data['das_spectrum_1d'], data['music_spectrum_1d'], data['capon_spectrum_1d'], data['true_angle_deg_1d'])
                
                self.canvas_fw_test.draw()
                self.fw_testbed_status_label.config(text="Status: Test Complete")
            
            self.full_wave_testbed_thread = None 

        else: 
            self.fw_testbed_status_label.config(text="Status: Running...")
            self.after(100, self._check_full_wave_testbed_status)

    def _saft_das_beamforming_for_fw_testbed(self, mimo_matrix_orig, p, grid_x, grid_y, layer_params, skin_surface_y_plot_coord,
                                          num_tx, num_rx, d_mm, f_ghz, tx_pos_x, rx_pos_x):
        tx_window = np.hanning(num_tx); rx_window = np.hanning(num_rx)
        window_2d = np.outer(tx_window, rx_window); mimo_matrix = mimo_matrix_orig * window_2d
        
        image = np.zeros((len(grid_x), len(grid_y)), dtype=np.complex128)

        for i, x in enumerate(grid_x):
            for j, y in enumerate(grid_y):
                pixel_pos = np.array([x, y])
                focused_sum = 0
                for tx_idx in range(num_tx):
                    for rx_idx in range(num_rx):
                        tx_antenna_pos = np.array([tx_pos_x[tx_idx], Y_ANTENNA_PLANE])
                        rx_antenna_pos = np.array([rx_pos_x[rx_idx], Y_ANTENNA_PLANE])
                        
                        path_metrics = self._calculate_layered_path_metrics(
                            tx_antenna_pos, pixel_pos, layer_params, f_ghz, skin_surface_y_plot_coord
                        )
                        _, phase_tx_pixel, _ = path_metrics

                        path_metrics = self._calculate_layered_path_metrics(
                            pixel_pos, rx_antenna_pos, layer_params, f_ghz, skin_surface_y_plot_coord
                        )
                        _, phase_pixel_rx, _ = path_metrics
                        
                        total_phase_correction = phase_tx_pixel + phase_pixel_rx
                        
                        focused_sum += mimo_matrix[tx_idx, rx_idx] * np.exp(1j * total_phase_correction)
                image[i, j] = focused_sum
        return image

    def calculate_effective_permittivity(self, layer_params, tumor_depth):
        total_dist_in_layers = 0; weighted_refractive_index_sum = 0
        for layer in layer_params:
            thickness_in_path = min(layer['thickness'], tumor_depth - total_dist_in_layers)
            if thickness_in_path <= 0: break
            weighted_refractive_index_sum += thickness_in_path * np.sqrt(layer['permit']); total_dist_in_layers += thickness_in_path
        if total_dist_in_layers == 0: return 1.0
        avg_refractive_index = weighted_refractive_index_sum / total_dist_in_layers
        return avg_refractive_index**2

    def saft_das_beamforming(self, mimo_matrix_orig, p, grid_x, grid_y, epsilon_eff):
        num_tx, num_rx = int(p['num_tx']), int(p['num_rx']); d = p['antenna_spacing']; f_ghz = p['freq_max']
        tx_window = np.hanning(num_tx); rx_window = np.hanning(num_rx); window_2d = np.outer(tx_window, rx_window); mimo_matrix = mimo_matrix_orig * window_2d
        fc = f_ghz * 1e9; wavelength_vacuum_mm = (c / fc) * 1000; k_tissue = (2 * np.pi / wavelength_vacuum_mm) * np.sqrt(epsilon_eff)
        tx_pos_x = np.arange(num_tx) * d - (num_tx - 1) * d / 2; rx_pos_x = np.arange(num_rx) * d - (num_rx - 1) * d / 2
        image = np.zeros((len(grid_x), len(grid_y)), dtype=np.complex128)
        for i, x in enumerate(grid_x):
            for j, y in enumerate(grid_y):
                if y >= -0.5: continue
                pixel_pos = np.array([x, y]); focused_sum = 0
                for tx_idx in range(num_tx):
                    for rx_idx in range(num_rx):
                        tx_pos = np.array([tx_pos_x[tx_idx], Y_ANTENNA_PLANE]); rx_pos = np.array([rx_pos_x[rx_idx], Y_ANTENNA_PLANE])
                        total_dist = np.linalg.norm(tx_pos - pixel_pos) + np.linalg.norm(rx_pos - pixel_pos)
                        phase_correction = np.exp(1j * k_tissue * total_dist)
                        focused_sum += mimo_matrix[tx_idx, rx_idx] * phase_correction
                image[i, j] = focused_sum
        return np.abs(image)**2

    def save_to_csv(self):
        if not self.last_run_data: messagebox.showinfo("No Data", "Please run a simulation first."); return
        filepath = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")], title="Save MIMO Matrix Data")
        if not filepath: return
        try:
            matrix = self.last_run_data['mimo_matrix']
            with open(filepath, 'w', newline='') as f:
                writer = csv.writer(f); writer.writerow([f"Rx_{i}_Mag" for i in range(matrix.shape[1])] + [f"Rx_{i}_Phase" for i in range(matrix.shape[1])])
                for tx_idx in range(matrix.shape[0]):
                    magnitudes = np.abs(matrix[tx_idx, :]); phases = np.angle(matrix[tx_idx, :]); writer.writerow(list(magnitudes) + list(phases))
            messagebox.showinfo("Success", f"Data successfully saved to {filepath}")
        except Exception as e: messagebox.showerror("Error Saving File", str(e))

    def plot_all_empty(self):
        self.ax_scenario.set_title("1. Scenario Geometry"); self.ax_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_mimo_matrix.set_title("2. MIMO Channel Matrix (Magnitude)")
        self.ax_mimo_matrix.grid(False); self.ax_saft_diag.set_title("3. SAFT/DAS Focusing Diagnostic"); self.ax_saft_diag.grid(True, linestyle='--', alpha=0.6)
        self.ax_saft_img.set_title("4. Reconstructed SAFT/DAS Image"); self.ax_saft_img.grid(False); self.ax_spectrum.set_title("5. Received Frequency Spectrum")
        self.ax_spectrum.grid(True, linestyle='--', alpha=0.6); self.canvas.draw()

    def plot_mimo_matrix(self, mimo_matrix):
        if self.cbar_mimo is not None:
            self.cbar_mimo.remove()
            self.cbar_mimo = None
        self.ax_mimo_matrix.clear(); im = self.ax_mimo_matrix.imshow(np.abs(mimo_matrix), cmap='viridis', aspect='auto')
        self.ax_mimo_matrix.set_title("2. MIMO Channel Matrix (Magnitude)"); self.ax_mimo_matrix.set_xlabel("Receiver Index"); self.ax_mimo_matrix.set_ylabel("Transmitter Index")
        self.cbar_mimo = self.fig.colorbar(im, ax=self.ax_mimo_matrix, label='Signal Magnitude (a.u.)', pad=0.01)

    def plot_saft_diagnostic(self, mimo_matrix, p, layer_params):
        self.ax_saft_diag.clear()
        if not p['tumor_present']: 
            self.ax_saft_diag.set_title("3. SAFT/DAS Diagnostic (No Tumor)")
            self.ax_saft_diag.grid(True, linestyle='--', alpha=0.6)
            return
            
        num_tx, num_rx = int(p['num_tx']), int(p['num_rx']); d = p['antenna_spacing']; f_ghz = p['freq_max']
        central_tx_idx = num_tx // 2; received_phase = np.unwrap(np.angle(mimo_matrix[central_tx_idx, :])); receivers = np.arange(num_rx)
        self.ax_saft_diag.plot(receivers, received_phase, 'o-', color='tab:blue', label='Actual Received Phase')
        
        tx_pos_x_arr = np.arange(num_tx) * d - (num_tx - 1) * d / 2
        rx_pos_x_arr = np.arange(num_rx) * d - (num_rx - 1) * d / 2
        tx_pos_source = np.array([tx_pos_x_arr[central_tx_idx], Y_ANTENNA_PLANE])
        pixel_pos = np.array([p['tumor_x'], p['tumor_y']])
        skin_surface_y_plot = Y_ANTENNA_PLANE - p['skin_surface_offset']

        correction_phase_rad = np.zeros(num_rx)
        for rx_idx in range(num_rx):
            rx_pos_detector = np.array([rx_pos_x_arr[rx_idx], Y_ANTENNA_PLANE])
            
            _, phase_tx_pixel, _ = self._calculate_layered_path_metrics(
                tx_pos_source, pixel_pos, layer_params, f_ghz, skin_surface_y_plot
            )
            _, phase_pixel_rx, _ = self._calculate_layered_path_metrics(
                pixel_pos, rx_pos_detector, layer_params, f_ghz, skin_surface_y_plot
            )
            correction_phase_rad[rx_idx] = phase_tx_pixel + phase_pixel_rx

        calculated_lag_phase = -np.unwrap(correction_phase_rad % (2 * np.pi)); phase_offset = np.mean(received_phase - calculated_lag_phase)
        self.ax_saft_diag.plot(receivers, calculated_lag_phase + phase_offset, 'x--', color='tab:orange', label='Calculated Focusing Phase')
        self.ax_saft_diag.set_title("3. SAFT/DAS Focusing Diagnostic"); self.ax_saft_diag.set_xlabel("Receiver Index"); self.ax_saft_diag.set_ylabel("Phase (radians)")
        self.ax_saft_diag.legend(bbox_to_anchor=(1.05, 1), loc='upper left'); self.ax_saft_diag.grid(True, linestyle='--', alpha=0.6)

    def plot_saft_image(self, grid_x, grid_y, saft_image, p):
        if self.cbar_das is not None:
            self.cbar_das.remove()
            self.cbar_das = None
        self.ax_saft_img.clear()
        image_db = 10 * np.log10(saft_image / (np.max(saft_image) + 1e-20) + 1e-20)
        im = self.ax_saft_img.imshow(image_db.T, cmap='plasma', extent=[grid_x[0], grid_x[-1], grid_y[-1], grid_y[0]], aspect='auto', interpolation='bilinear', vmin=-30, vmax=0)
        if p['tumor_present']:
            max_idx = np.unravel_index(np.argmax(saft_image), saft_image.shape); est_x, est_y = grid_x[max_idx[0]], grid_y[max_idx[1]]
            self.ax_saft_img.scatter(p['tumor_x'], p['tumor_y'], s=200, facecolors='none', edgecolors='lime', marker='o', linewidth=2, label=f'Ground Truth')
            self.ax_saft_img.scatter(est_x, est_y, s=250, c='white', marker='x', linewidth=2, label=f'Estimated Target')
        self.ax_saft_img.set_title("4. Reconstructed SAFT/DAS Image"); self.ax_saft_img.set_xlabel("X Position (mm)"); self.ax_saft_img.set_ylabel("Y Position (mm)")
        self.ax_saft_img.legend(bbox_to_anchor=(1.05, 1), loc='upper left'); 
        self.cbar_das = self.fig.colorbar(im, ax=self.ax_saft_img, label='Reflectivity (dB)', pad=0.01)

    def plot_scenario(self, p, layer_params):
        self.ax_scenario.clear(); 
        max_tx_rx = max(p.get('num_tx', 4), p.get('num_rx', 16)); max_width = (max_tx_rx + 4) * p.get('antenna_spacing', 2.0);
        
        current_y_plot = Y_ANTENNA_PLANE - p['skin_surface_offset'] 
        
        if layer_params:
            for i, layer in enumerate(layer_params):
                self.ax_scenario.add_patch(plt.Rectangle((-max_width/2, current_y_plot - layer['thickness']), 
                                                         max_width, layer['thickness'], 
                                                         edgecolor='black', facecolor=f'C{i}', alpha=0.5, 
                                                         label=f'Layer {i+1} (εᵣ={layer["permit"]:.1f})'))
                current_y_plot -= layer['thickness']
        
        if p['tumor_present']: 
            self.ax_scenario.add_patch(plt.Circle((p['tumor_x'], p['tumor_y']), p['tumor_radius'], 
                                                  color='red', label=f'Tumor (εᵣ={p["tumor_permit"]:.1f})'))
        
        num_tx, num_rx, d = int(p['num_tx']), int(p['num_rx']), p['antenna_spacing']
        tx_pos_x = np.arange(num_tx) * d - (num_tx - 1) * d / 2
        rx_pos_x = np.arange(num_rx) * d - (num_rx - 1) * d / 2
        
        self.ax_scenario.plot(tx_pos_x, np.full(num_tx, Y_ANTENNA_PLANE), 'gv', markersize=8, linestyle='None', label='Tx')
        self.ax_scenario.plot(rx_pos_x, np.full(num_rx, Y_ANTENNA_PLANE), 'kv', markersize=6, linestyle='None', label='Rx')
        
        self.ax_scenario.set_title("1. Scenario Geometry"); self.ax_scenario.set_xlabel("X (mm)"); self.ax_scenario.set_ylabel("Y (mm)")
        self.ax_scenario.legend(bbox_to_anchor=(1.05, 1), loc='upper left'); self.ax_scenario.axis('equal'); self.ax_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_scenario.set_xlim(-max_width/2, max_width/2)
        
        min_y_bound = min(p['tumor_y'] - p['tumor_radius'] - 5 if p['tumor_present'] else Y_ANTENNA_PLANE, current_y_plot - 5)
        max_y_bound = Y_ANTENNA_PLANE + 5 
        self.ax_scenario.set_ylim(min_y_bound, max_y_bound)


    def plot_spectrum(self, spectrum_data, freq_axis):
        self.ax_spectrum.clear()
        if spectrum_data.get('healthy') is not None: self.ax_spectrum.plot(freq_axis, 20*np.log10(np.abs(spectrum_data['healthy']) + 1e-20), label='Healthy Signal')
        if spectrum_data.get('unhealthy') is not None:
            self.ax_spectrum.plot(freq_axis, 20*np.log10(np.abs(spectrum_data['unhealthy']) + 1e-20), label='Unhealthy Signal')
            diff_spectrum = np.abs(spectrum_data['unhealthy'] - spectrum_data['healthy'])
            self.ax_spectrum.plot(freq_axis, 20*np.log10(diff_spectrum + 1e-20), label='Differential', linestyle='--')
        self.ax_spectrum.set_title("5. Received Frequency Spectrum"); self.ax_spectrum.set_xlabel("Frequency (GHz)"); self.ax_spectrum.set_ylabel("Magnitude (dB)")
        self.ax_spectrum.grid(True, linestyle='--', alpha=0.6); 
        self.ax_spectrum.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    def plot_test_all_empty(self):
        self.ax_test_scenario.clear(); self.ax_test_scenario.set_title("Analytical Testbed Scenario"); self.ax_test_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_test_scenario.set_aspect('equal', adjustable='box')
        self.ax_test_das_2d.clear(); self.ax_test_das_2d.set_title("Analytical DAS Range-Angle Image"); self.ax_test_das_2d.grid(False); self.ax_test_das_2d.set_aspect('equal', adjustable='box')
        self.ax_test_angular_spectrum.clear(); self.ax_test_angular_spectrum.set_title("Analytical Angular Spectrum Comparison"); self.ax_test_angular_spectrum.grid(True, linestyle='--', alpha=0.6)
        self.canvas_test.draw()

    def plot_test_scenario(self, targets, layers, tx_pos_x, rx_pos_x, skin_surface_y_plot):
        self.ax_test_scenario.clear()
        
        array_max_x = max(tx_pos_x[-1] if tx_pos_x.size > 0 else 0, rx_pos_x[-1] if rx_pos_x.size > 0 else 0)
        array_min_x = min(tx_pos_x[0] if tx_pos_x.size > 0 else 0, rx_pos_x[0] if rx_pos_x.size > 0 else 0)
        max_array_width = array_max_x - array_min_x
        plot_width = max_array_width + 20 
        
        current_y_plot = skin_surface_y_plot
        if layers:
            for i, layer in enumerate(layers):
                self.ax_test_scenario.add_patch(plt.Rectangle((-plot_width/2, current_y_plot - layer['thickness']), 
                                                         plot_width, layer['thickness'], 
                                                         edgecolor='black', facecolor=f'C{i}', alpha=0.5, 
                                                         label=f'Layer {i+1}'))
        
        if targets:
            for i, target in enumerate(targets):
                self.ax_test_scenario.add_patch(plt.Circle((target['x'], target['y']), target['radius'], 
                                                          color='red', alpha=0.7, label=f'Target {i+1}'))
        
        if tx_pos_x.size > 0:
            self.ax_test_scenario.plot(tx_pos_x, np.full(len(tx_pos_x), Y_ANTENNA_PLANE), 'gv', markersize=8, linestyle='None', label='Tx')
        if rx_pos_x.size > 0:
            self.ax_test_scenario.plot(rx_pos_x, np.full(len(rx_pos_x), Y_ANTENNA_PLANE), 'kv', markersize=6, linestyle='None', label='Rx')
        
        self.ax_test_scenario.set_title("Analytical Testbed Scenario"); self.ax_test_scenario.set_xlabel("X (mm)"); self.ax_test_scenario.set_ylabel("Y (mm)")
        self.ax_test_scenario.legend(bbox_to_anchor=(-0.1, 1.0), loc='upper right')
        self.ax_test_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_test_scenario.set_aspect('equal', adjustable='box')
        
        min_target_y = min([t['y'] - t['radius'] for t in targets]) if targets else Y_ANTENNA_PLANE 
        min_layer_y = current_y_plot if layers else Y_ANTENNA_PLANE
        min_y_bound = min(min_target_y, min_layer_y) - 5 
        max_y_bound = Y_ANTENNA_PLANE + 5 
        self.ax_test_scenario.set_ylim(min_y_bound, max_y_bound)
        self.ax_test_scenario.set_xlim(-plot_width/2, plot_width/2)

    def plot_test_das_2d_image(self, image_x_grid, image_y_grid, das_image_2d, targets, skin_surface_y_plot):
        if self.cbar_test_das_2d is not None:
            self.cbar_test_das_2d.remove() 
            self.cbar_test_das_2d = None

        self.ax_test_das_2d.clear()
        
        image_db = 10 * np.log10(das_image_2d / (np.max(das_image_2d) + 1e-20) + 1e-20)
        
        im = self.ax_test_das_2d.imshow(image_db.T, cmap='plasma', 
                                         extent=[image_x_grid[0], image_x_grid[-1], image_y_grid[-1], image_y_grid[0]], 
                                         aspect='auto', interpolation='bilinear', vmin=-30, vmax=0)
        
        if targets:
            for target in targets:
                self.ax_test_das_2d.scatter(target['x'], target['y'], s=200, facecolors='none', edgecolors='lime', marker='o', linewidth=2, label=f'GT Target')
        
        self.ax_test_das_2d.axhline(y=skin_surface_y_plot, color='cyan', linestyle='--', label='Skin Surface')

        self.ax_test_das_2d.set_title("Analytical DAS Range-Angle Image"); self.ax_test_das_2d.set_xlabel("X (mm)"); self.ax_test_das_2d.set_ylabel("Y (mm)")
        self.ax_test_das_2d.legend(bbox_to_anchor=(1.05, 1), loc='upper left'); 
        self.cbar_test_das_2d = self.fig_test_plots.colorbar(im, ax=self.ax_test_das_2d, label='Reflectivity (dB)', pad=0.01)
        self.ax_test_das_2d.set_aspect('equal', adjustable='box')


    def plot_test_angular_spectrum(self, angles_deg, das_spectrum, music_spectrum, capon_spectrum, true_angle_deg): 
        self.ax_test_angular_spectrum.clear()
        
        self.ax_test_angular_spectrum.plot(angles_deg, das_spectrum, label='Delay-and-Sum (DAS)')
        self.ax_test_angular_spectrum.plot(angles_deg, music_spectrum, label='MUSIC', linestyle='--')
        self.ax_test_angular_spectrum.plot(angles_deg, capon_spectrum, label='Capon', linestyle=':') 
        
        self.ax_test_angular_spectrum.axvline(x=true_angle_deg, color='r', linestyle=':', label=f'True Angle ({true_angle_deg:.1f}°)')
        self.ax_test_angular_spectrum.set_title("Analytical Angular Spectrum Comparison"); self.ax_test_angular_spectrum.grid(True, linestyle='--', alpha=0.6); self.ax_test_angular_spectrum.set_xlabel("Angle (Degrees)")
        self.ax_test_angular_spectrum.set_ylabel("Normalized Power"); 
        self.ax_test_angular_spectrum.legend(bbox_to_anchor=(1.02, 1), loc='upper left'); self.ax_test_angular_spectrum.set_ylim(0, 1.1)


    def plot_fw_all_empty(self):
        self.ax_fw_scenario.clear(); self.ax_fw_scenario.set_title("FDTD Testbed Scenario"); self.ax_fw_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_fw_scenario.set_aspect('equal', adjustable='box')
        self.ax_fw_das_2d.clear(); self.ax_fw_das_2d.set_title("FDTD DAS Range-Angle Image"); self.ax_fw_das_2d.grid(False); self.ax_fw_das_2d.set_aspect('equal', adjustable='box')
        self.ax_fw_angular_spectrum.clear(); self.ax_fw_angular_spectrum.set_title("FDTD Angular Spectrum Comparison"); self.ax_fw_angular_spectrum.grid(True, linestyle='--', alpha=0.6)
        self.canvas_fw_test.draw()

    def plot_fw_scenario(self, targets, layers, tx_pos_x, rx_pos_x, skin_surface_y_plot):
        self.ax_fw_scenario.clear()
        
        array_max_x = max(tx_pos_x[-1] if tx_pos_x.size > 0 else 0, rx_pos_x[-1] if rx_pos_x.size > 0 else 0)
        array_min_x = min(tx_pos_x[0] if tx_pos_x.size > 0 else 0, rx_pos_x[0] if rx_pos_x.size > 0 else 0)
        max_array_width = array_max_x - array_min_x
        plot_width = max_array_width + 20 
        
        current_y_plot = skin_surface_y_plot
        if layers:
            for i, layer in enumerate(layers):
                self.ax_fw_scenario.add_patch(plt.Rectangle((-plot_width/2, current_y_plot - layer['thickness']), 
                                                         plot_width, layer['thickness'], 
                                                         edgecolor='black', facecolor=f'C{i}', alpha=0.5, 
                                                         label=f'Layer {i+1}'))
        
        if targets:
            for i, target in enumerate(targets):
                self.ax_fw_scenario.add_patch(plt.Circle((target['x'], target['y']), target['radius'], 
                                                          color='red', alpha=0.7, label=f'Target {i+1}'))
        
        if tx_pos_x.size > 0:
            self.ax_fw_scenario.plot(tx_pos_x, np.full(len(tx_pos_x), Y_ANTENNA_PLANE), 'gv', markersize=8, linestyle='None', label='Tx')
        if rx_pos_x.size > 0:
            self.ax_fw_scenario.plot(rx_pos_x, np.full(len(rx_pos_x), Y_ANTENNA_PLANE), 'kv', markersize=6, linestyle='None', label='Rx')
        
        self.ax_fw_scenario.set_title("FDTD Testbed Scenario"); self.ax_fw_scenario.set_xlabel("X (mm)"); self.ax_fw_scenario.set_ylabel("Y (mm)")
        self.ax_fw_scenario.legend(bbox_to_anchor=(-0.1, 1.0), loc='upper right')
        self.ax_fw_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_fw_scenario.set_aspect('equal', adjustable='box')
        
        min_target_y = min([t['y'] - t['radius'] for t in targets]) if targets else Y_ANTENNA_PLANE 
        min_layer_y = current_y_plot if layers else Y_ANTENNA_PLANE
        min_y_bound = min(min_target_y, min_layer_y) - 5 
        max_y_bound = Y_ANTENNA_PLANE + 5 
        self.ax_fw_scenario.set_ylim(min_y_bound, max_y_bound)
        self.ax_fw_scenario.set_xlim(-plot_width/2, plot_width/2)

    def plot_fw_das_2d_image(self, image_x_grid, image_y_grid, das_image_2d, targets, skin_surface_y_plot):
        if self.cbar_fw_das_2d is not None:
            self.cbar_fw_das_2d.remove() 
            self.cbar_fw_das_2d = None

        self.ax_fw_das_2d.clear()
        
        image_db = 10 * np.log10(das_image_2d / (np.max(das_image_2d) + 1e-20) + 1e-20)
        
        im = self.ax_fw_das_2d.imshow(image_db.T, cmap='plasma', 
                                         extent=[image_x_grid[0], image_x_grid[-1], image_y_grid[-1], image_y_grid[0]], 
                                         aspect='auto', interpolation='bilinear', vmin=-30, vmax=0)
        
        if targets:
            for target in targets:
                self.ax_fw_das_2d.scatter(target['x'], target['y'], s=200, facecolors='none', edgecolors='lime', marker='o', linewidth=2, label=f'GT Target')
        
        self.ax_fw_das_2d.axhline(y=skin_surface_y_plot, color='cyan', linestyle='--', label='Skin Surface')

        self.ax_fw_das_2d.set_title("FDTD DAS Range-Angle Image"); self.ax_fw_das_2d.set_xlabel("X (mm)"); self.ax_fw_das_2d.set_ylabel("Y (mm)")
        self.ax_fw_das_2d.legend(bbox_to_anchor=(1.05, 1), loc='upper left'); 
        self.cbar_fw_das_2d = self.fig_fw_test_plots.colorbar(im, ax=self.ax_fw_das_2d, label='Reflectivity (dB)', pad=0.01)
        self.ax_fw_das_2d.set_aspect('equal', adjustable='box')


    def plot_fw_angular_spectrum(self, angles_deg, das_spectrum, music_spectrum, capon_spectrum, true_angle_deg): 
        self.ax_fw_angular_spectrum.clear()
        
        self.ax_fw_angular_spectrum.plot(angles_deg, das_spectrum, label='Delay-and-Sum (DAS)')
        self.ax_fw_angular_spectrum.plot(angles_deg, music_spectrum, label='MUSIC', linestyle='--')
        self.ax_fw_angular_spectrum.plot(angles_deg, capon_spectrum, label='Capon', linestyle=':') 
        
        self.ax_fw_angular_spectrum.axvline(x=true_angle_deg, color='r', linestyle=':', label=f'True Angle ({true_angle_deg:.1f}°)')
        self.ax_fw_angular_spectrum.set_title("FDTD Angular Spectrum Comparison"); self.ax_fw_angular_spectrum.grid(True, linestyle='--', alpha=0.6); self.ax_fw_angular_spectrum.set_xlabel("Angle (Degrees)")
        self.ax_fw_angular_spectrum.set_ylabel("Normalized Power"); 
        self.ax_fw_angular_spectrum.legend(bbox_to_anchor=(1.02, 1), loc='upper left'); self.ax_fw_angular_spectrum.set_ylim(0, 1.1)


    def create_testbed_v2_fdtd_tab(self):
        main_frame = ttk.Frame(self.tab4, padding="5", style='App.TFrame')
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        control_outer_frame, scrollable_inner_frame = self._create_scrollable_param_frame(main_frame, text_label="TestbedV2 Parameters")
        control_outer_frame.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)

        plot_frame = ttk.Frame(main_frame, padding="5", style='App.TFrame')
        plot_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        self.testbed_v2_params = {}

        ttk.Label(scrollable_inner_frame, text="--- MIMO Antenna Array (Common) ---", style='Header.TLabel').pack(pady=5, anchor='w')
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_num_tx", "Num Transmitters:", 4) 
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_num_rx", "Num Receivers:", 16) 
        
        self.v2_freq_var = self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_freq", "Center Frequency (GHz):", 77, return_var=True) 
        spacing_frame_v2 = ttk.Frame(scrollable_inner_frame); spacing_frame_v2.pack(fill=tk.X, pady=2)
        ttk.Checkbutton(spacing_frame_v2, text="Auto-calculate spacing (λ/2)", variable=self.v2_auto_spacing_var, command=self.toggle_testbed_v2_spacing_entry).pack(anchor='w', padx=5)
        returned_widgets_v2 = self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_spacing", "Antenna Spacing (mm):", 1.95, return_var_and_widget=True) 
        self.v2_spacing_entry = returned_widgets_v2['widget']
        self.v2_freq_var.trace("w", self.update_testbed_v2_spacing)
        
        ttk.Label(scrollable_inner_frame, text="--- System Parameters (Analytical) ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_tx_power_dbm", "Tx Power (dBm):", 10.0) 
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_rx_gain_db", "Rx Gain (dB):", 20.0) 
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_noise_figure_db", "Noise Figure (dB):", 3.0) 
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_bandwidth_mhz", "Bandwidth (MHz):", 200.0) 
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_system_temp_k", "System Temp (K):", 290.0) 

        ttk.Label(scrollable_inner_frame, text="--- Medium Properties (Common) ---", style='Header.TLabel').pack(pady=10, anchor='w')
        ttk.Checkbutton(scrollable_inner_frame, text="Enable Skin Layers", variable=self.v2_layers_enabled_var, command=self.toggle_testbed_v2_layer_entries).pack(anchor='w', padx=5)
        v2_skin_offset_info = self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_skin_surface_offset", "Skin Surface Offset (mm):", 2.5, return_var_and_widget=True) 
        self.v2_skin_offset_entry_widget = v2_skin_offset_info['widget']
        
        self.v2_layer_entries = []
        v2_num_layers_info = self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_num_layers", "Num Layers:", 2, return_var_and_widget=True) 
        v2_num_layers_var = v2_num_layers_info['var']
        self.v2_num_layers_entry_widget = v2_num_layers_info['widget']
        
        outer_layer_scroll_frame_v2, self.v2_layer_params_frame = self._create_scrollable_param_frame(scrollable_inner_frame, text_label="Skin Layers Properties", min_canvas_height=200)
        outer_layer_scroll_frame_v2.pack(fill=tk.X, expand=True, pady=5, padx=5)
        self._create_dynamic_layer_widgets(self.v2_layer_params_frame, v2_num_layers_var, self.v2_layer_entries)
        
        ttk.Label(scrollable_inner_frame, text="--- Multiple Targets (Common) ---", style='Header.TLabel').pack(pady=10, anchor='w')
        ttk.Checkbutton(scrollable_inner_frame, text="Enable Multiple Targets", variable=self.v2_targets_enabled_var, command=self.toggle_testbed_v2_target_entries).pack(anchor='w', padx=5)

        self.v2_target_entries = []
        v2_num_targets_info = self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_num_targets", "Number of Targets:", 1, return_var_and_widget=True) 
        v2_num_targets_var = v2_num_targets_info['var']
        self.v2_num_targets_entry_widget = v2_num_targets_info['widget']
        
        outer_target_scroll_frame_v2, self.v2_target_params_frame = self._create_scrollable_param_frame(scrollable_inner_frame, text_label="Target Properties", min_canvas_height=200)
        outer_target_scroll_frame_v2.pack(fill=tk.X, expand=True, pady=5, padx=5)
        self._create_dynamic_target_widgets(self.v2_target_params_frame, v2_num_targets_var, self.v2_target_entries) 

        ttk.Label(scrollable_inner_frame, text="--- FDTD Simulation Control ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_resolution", "Sim Resolution (pixels/mm):", 15) 
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_pml_thickness", "PML Thickness (mm):", 2.0) 

        ttk.Label(scrollable_inner_frame, text="--- Imaging Parameters (Common) ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_image_x_min", "Image X Min (mm):", -20) 
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_image_x_max", "Image X Max (mm):", 20) 
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_image_y_min", "Image Y Min (mm):", -40) 
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_image_y_max", "Image Y Max (mm):", -0.5) 
        self.add_param(scrollable_inner_frame, self.testbed_v2_params, "v2_image_resolution_mm", "Image Resolution (mm):", 1.0) 

        run_options_frame = ttk.LabelFrame(scrollable_inner_frame, text="Run Options", padding="10")
        run_options_frame.pack(pady=10, fill=tk.X, padx=5)
        ttk.Checkbutton(run_options_frame, text="Run Analytical Simulation (for comparison)", variable=self.run_fdtd_with_analytical_var).pack(anchor='w', padx=5) 

        self.run_testbed_v2_button = ttk.Button(scrollable_inner_frame, text="Run FDTD Testbed (and Analytical if checked)", command=self.run_testbed_v2_fdtd)
        self.run_testbed_v2_button.pack(pady=10, fill=tk.X, padx=5)
        self.v2_testbed_progress_bar = ttk.Progressbar(scrollable_inner_frame, mode='determinate')
        self.v2_testbed_progress_bar.pack(pady=2, fill=tk.X, padx=5)
        self.v2_testbed_status_label = ttk.Label(scrollable_inner_frame, text="Status: Idle")
        self.v2_testbed_status_label.pack(pady=2, fill=tk.X, padx=5) 

        self.update_testbed_v2_spacing() 

        self.fig_v2_test_plots = plt.figure(figsize=(14, 10)) 
        gs_v2 = self.fig_v2_test_plots.add_gridspec(3, 2, height_ratios=[1.2, 1, 1], hspace=0.5, wspace=0.3) 

        self.ax_v2_scenario = self.fig_v2_test_plots.add_subplot(gs_v2[0, :]) 
        self.ax_v2_das_2d_analytical = self.fig_v2_test_plots.add_subplot(gs_v2[1, 0])
        self.ax_v2_das_2d_fdtd = self.fig_v2_test_plots.add_subplot(gs_v2[1, 1])
        self.ax_v2_angular_analytical = self.fig_v2_test_plots.add_subplot(gs_v2[2, 0])
        self.ax_v2_angular_fdtd = self.fig_v2_test_plots.add_subplot(gs_v2[2, 1])

        self.canvas_v2_test = FigureCanvasTkAgg(self.fig_v2_test_plots, master=plot_frame); self.canvas_v2_test.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        self.plot_testbed_v2_all_empty()

    def toggle_testbed_v2_spacing_entry(self):
        self.v2_spacing_entry.config(state='disabled' if self.v2_auto_spacing_var.get() else 'normal')
        self.update_testbed_v2_spacing()

    def update_testbed_v2_spacing(self, *args):
        if self.v2_auto_spacing_var.get():
            try:
                freq_ghz = self.testbed_v2_params['v2_freq'].get()
                if freq_ghz > 0:
                    lambda_mm = (c / (freq_ghz * 1e9)) * 1000
                    self.testbed_v2_params['v2_spacing'].set(round(lambda_mm / 2, 4))
            except (tk.TclError, KeyError): pass

    def toggle_testbed_v2_layer_entries(self):
        self._toggle_dynamic_widget_frame(self.v2_layers_enabled_var, self.v2_num_layers_entry_widget, self.v2_layer_params_frame)

    def toggle_testbed_v2_target_entries(self):
        self._toggle_dynamic_widget_frame(self.v2_targets_enabled_var, self.v2_num_targets_entry_widget, self.v2_target_params_frame)

    def get_testbed_v2_layer_params(self, entries_list): 
        return self.get_layer_params(entries_list)
        
    def get_testbed_v2_target_params(self, entries_list): 
        return [{'x': e['x'].get(), 'y': e['y'].get(), 'radius': e['radius'].get(), 'rcs': e['rcs'].get(), 'permit': e['permit'].get(), 'conductivity': e['conductivity'].get()} for e in entries_list]
    
    def run_testbed_v2_fdtd(self):
        try:
            if self.testbed_v2_thread and self.testbed_v2_thread.is_alive():
                messagebox.showwarning("Testbed Running", "A TestbedV2 simulation is already in progress.")
                return

            try:
                p_v2_check = {name: var.get() for name, var in self.testbed_v2_params.items()}

                if p_v2_check['v2_freq'] <= 0 or p_v2_check['v2_num_rx'] <= 0 or p_v2_check['v2_num_tx'] <= 0 or \
                   p_v2_check['v2_image_resolution_mm'] <= 0:
                    messagebox.showerror("Input Error", "Common array/imaging parameters must be positive.")
                    return
                
                if self.v2_layers_enabled_var.get() and p_v2_check['v2_skin_surface_offset'] <= 0:
                    messagebox.showerror("Input Error", "Skin Surface Offset must be positive if layers are enabled.")
                    return

                if self.v2_targets_enabled_var.get() and p_v2_check['v2_num_targets'] <= 0:
                    messagebox.showerror("Input Error", "Number of targets must be positive if enabled.")
                    return
                
                if p_v2_check['v2_image_x_min'] >= p_v2_check['v2_image_x_max'] or p_v2_check['v2_image_y_min'] >= p_v2_check['v2_image_y_max']:
                    messagebox.showerror("Input Error", "Image X/Y Max must be greater than Min.")
                    return

                if self.run_fdtd_with_analytical_var.get():
                    if p_v2_check['v2_tx_power_dbm'] < -100 or p_v2_check['v2_rx_gain_db'] < -50 or p_v2_check['v2_noise_figure_db'] < 0 or \
                       p_v2_check['v2_bandwidth_mhz'] <= 0 or p_v2_check['v2_system_temp_k'] <= 0:
                        messagebox.showerror("Input Error", "Analytical system parameters must be valid (e.g., positive for bandwidth/temp, reasonable for power/gain/NF).")
                        return

                if p_v2_check['v2_resolution'] <= 0 or p_v2_check['v2_pml_thickness'] <= 0:
                    messagebox.showerror("Input Error", "FDTD resolution and PML thickness must be positive.")
                    return

            except (tk.TclError, ValueError) as e:
                messagebox.showerror("Input Error", f"Invalid numerical input provided: {e}")
                return

            self.run_testbed_v2_button.config(state='disabled')
            self.testbed_v2_results = None
            self.v2_testbed_status_label.config(text="Status: Initializing TestbedV2...")
            
            if self.v2_testbed_progress_bar:
                self.v2_testbed_progress_bar['value'] = 0
                self.v2_testbed_progress_bar['maximum'] = 100 
                self.v2_testbed_progress_bar['mode'] = 'determinate'

            self.testbed_v2_thread = threading.Thread(target=self._execute_testbed_v2_fdtd_thread)
            self.testbed_v2_thread.daemon = True 
            self.testbed_v2_thread.start()
            self.after(100, self._check_testbed_v2_fdtd_status)

        except Exception as e:
            error_msg = f"An unexpected error occurred in run_testbed_v2_fdtd: {e}"
            messagebox.showerror("Critical Error", error_msg)
            self.run_testbed_v2_button.config(state='normal')
            self.v2_testbed_status_label.config(text="Status: Critical Error.")

    def _execute_testbed_v2_fdtd_thread(self):
        try:
            p_v2 = {name: var.get() for name, var in self.testbed_v2_params.items()}
            p_v2['v2_layers_enabled'] = self.v2_layers_enabled_var.get()
            p_v2['v2_targets_enabled'] = self.v2_targets_enabled_var.get() 

            all_results = {}
            total_tasks = 2 if self.run_fdtd_with_analytical_var.get() else 1
            progress_step_size = 100 / total_tasks
            current_overall_progress = 0

            if self.run_fdtd_with_analytical_var.get():
                self.v2_testbed_status_label.config(text="Status: Analytical Simulation Running...")
                
                analytical_p = {
                    'test_num_tx_antennas': p_v2['v2_num_tx'],
                    'test_num_rx_antennas': p_v2['v2_num_rx'],
                    'test_spacing': p_v2['v2_spacing'],
                    'test_freq': p_v2['v2_freq'],
                    'tx_power_dbm': p_v2['v2_tx_power_dbm'],
                    'rx_gain_db': p_v2['v2_rx_gain_db'],
                    'noise_figure_db': p_v2['v2_noise_figure_db'],
                    'bandwidth_mhz': p_v2['v2_bandwidth_mhz'],
                    'system_temp_k': p_v2['v2_system_temp_k'],
                    'layers_enabled': p_v2['v2_layers_enabled'],
                    'test_skin_surface_offset': p_v2['v2_skin_surface_offset'],
                    'targets_enabled': p_v2['v2_targets_enabled'],
                    'image_x_min': p_v2['v2_image_x_min'],
                    'image_x_max': p_v2['v2_image_x_max'],
                    'image_y_min': p_v2['v2_image_y_min'],
                    'image_y_max': p_v2['v2_image_y_max'],
                    'image_resolution_mm': p_v2['v2_image_resolution_mm'],
                }

                analytical_res = self._perform_analytical_sim_and_beamforming(
                    analytical_p, self.v2_layer_entries, self.v2_target_entries, 
                    self.v2_testbed_status_label, self.v2_testbed_progress_bar, current_overall_progress
                )
                all_results['analytical'] = analytical_res
                current_overall_progress += progress_step_size
                if self.v2_testbed_progress_bar: self.v2_testbed_progress_bar['value'] = current_overall_progress
            
            self.v2_testbed_status_label.config(text="Status: FDTD Simulation Running...")
            
            fdtd_p = {
                'fw_num_tx': p_v2['v2_num_tx'],
                'fw_num_rx': p_v2['v2_num_rx'],
                'fw_antenna_spacing': p_v2['v2_spacing'],
                'fw_freq_max': p_v2['v2_freq'],
                'fw_resolution': p_v2['v2_resolution'],
                'fw_pml_thickness': p_v2['v2_pml_thickness'],
                'fw_layers_enabled': p_v2['v2_layers_enabled'],
                'fw_skin_surface_offset': p_v2['v2_skin_surface_offset'],
                'fw_targets_enabled': p_v2['v2_targets_enabled'],
                'fw_image_x_min': p_v2['v2_image_x_min'],
                'fw_image_x_max': p_v2['v2_image_x_max'],
                'fw_image_y_min': p_v2['v2_image_y_min'],
                'fw_image_y_max': p_v2['v2_image_y_max'],
                'fw_image_resolution_mm': p_v2['v2_image_resolution_mm'],
            }

            fdtd_res = self._perform_fdtd_sim_and_beamforming(
                fdtd_p, self.v2_layer_entries, self.v2_target_entries, 
                self.v2_testbed_status_label, self.v2_testbed_progress_bar, current_overall_progress
            )
            all_results['fdtd'] = fdtd_res
            current_overall_progress += progress_step_size
            if self.v2_testbed_progress_bar: self.v2_testbed_progress_bar['value'] = current_overall_progress

            self.testbed_v2_results = all_results
        except Exception as e: 
            import traceback
            self.testbed_v2_results = {"error": f"{e}\n{traceback.format_exc()}"}

    def _check_testbed_v2_fdtd_status(self):
        if self.testbed_v2_results:
            self.run_testbed_v2_button.config(state='normal')
            if self.v2_testbed_progress_bar: self.v2_testbed_progress_bar.stop()
            if "error" in self.testbed_v2_results: 
                messagebox.showerror("TestbedV2 Error", str(self.testbed_v2_results["error"]))
                self.v2_testbed_status_label.config(text="Status: Error occurred.")
            else:
                self.v2_testbed_status_label.config(text="Status: Test Complete")
                self.plot_testbed_v2_results(self.testbed_v2_results)
                self.canvas_v2_test.draw()

                results_text = "TestbedV2 Validation Metrics:\n\n"
                
                targets = self.testbed_v2_results['fdtd']['targets'] 
                true_target_x = targets[0]['x'] if targets else np.nan
                true_target_y = targets[0]['y'] if targets else np.nan
                
                if 'analytical' in self.testbed_v2_results and self.run_fdtd_with_analytical_var.get():
                    analytical_data = self.testbed_v2_results['analytical']
                    max_idx_analytical = np.unravel_index(np.argmax(analytical_data['das_image_2d']), analytical_data['das_image_2d'].shape)
                    est_x_analytical = analytical_data['image_x_grid'][max_idx_analytical[0]]
                    est_y_analytical = analytical_data['image_y_grid'][max_idx_analytical[1]]
                    
                    max_angle_idx_analytical = np.argmax(analytical_data['das_spectrum_1d'])
                    est_angle_analytical = analytical_data['angles_deg_1d'][max_angle_idx_analytical]

                    results_text += "Analytical Results:\n"
                    results_text += f"  - DAS 2D Peak: X={est_x_analytical:.2f}mm, Y={est_y_analytical:.2f}mm\n"
                    results_text += f"  - DAS 1D Angle Peak: {est_angle_analytical:.2f}°\n"
                
                if 'fdtd' in self.testbed_v2_results:
                    fdtd_data = self.testbed_v2_results['fdtd']
                    max_idx_fdtd = np.unravel_index(np.argmax(fdtd_data['das_image_2d']), fdtd_data['das_image_2d'].shape)
                    est_x_fdtd = fdtd_data['image_x_grid'][max_idx_fdtd[0]]
                    est_y_fdtd = fdtd_data['image_y_grid'][max_idx_fdtd[1]]

                    max_angle_idx_fdtd = np.argmax(fdtd_data['das_spectrum_1d'])
                    est_angle_fdtd = fdtd_data['angles_deg_1d'][max_angle_idx_fdtd]

                    results_text += "\nFDTD Results:\n"
                    results_text += f"  - DAS 2D Peak: X={est_x_fdtd:.2f}mm, Y={est_y_fdtd:.2f}mm\n"
                    results_text += f"  - DAS 1D Angle Peak: {est_angle_fdtd:.2f}°\n"

                if np.isfinite(true_target_x) and np.isfinite(true_target_y):
                    results_text += f"\nGround Truth Target: X={true_target_x:.2f}mm, Y={true_target_y:.2f}mm\n"
                    
                    if 'analytical' in self.testbed_v2_results and self.run_fdtd_with_analytical_var.get():
                        dist_analytical = np.sqrt((est_x_analytical - true_target_x)**2 + (est_y_analytical - true_target_y)**2)
                        results_text += f"  - Analytical 2D Error Distance: {dist_analytical:.2f}mm\n"
                        results_text += f"  - Analytical 1D Angle Error: {abs(est_angle_analytical - analytical_data['true_angle_deg_1d']):.2f}°\n"

                    if 'fdtd' in self.testbed_v2_results:
                        dist_fdtd = np.sqrt((est_x_fdtd - true_target_x)**2 + (est_y_fdtd - true_target_y)**2)
                        results_text += f"  - FDTD 2D Error Distance: {dist_fdtd:.2f}mm\n"
                        results_text += f"  - FDTD 1D Angle Error: {abs(est_angle_fdtd - fdtd_data['true_angle_deg_1d']):.2f}°\n"
                
                if 'analytical' in self.testbed_v2_results and 'fdtd' in self.testbed_v2_results and self.run_fdtd_with_analytical_var.get():
                    if np.isfinite(est_x_analytical) and np.isfinite(est_y_analytical) and np.isfinite(est_x_fdtd) and np.isfinite(est_y_fdtd):
                        diff_dist = np.sqrt((est_x_analytical - est_x_fdtd)**2 + (est_y_analytical - est_y_fdtd)**2)
                        results_text += f"\nAnalytical vs FDTD Comparison (Estimated Peaks):\n"
                        results_text += f"  - 2D Peak Location Difference: {diff_dist:.2f}mm\n"
                        results_text += f"  - 1D Angular Peak Difference: {abs(est_angle_analytical - est_angle_fdtd):.2f}°\n"
                
                messagebox.showinfo("TestbedV2 Complete", results_text)

            self.testbed_v2_thread = None 
        else: 
            self.v2_testbed_status_label.config(text="Status: Running...")
            self.after(100, self._check_testbed_v2_fdtd_status)

    def plot_testbed_v2_results(self, results):
        # self.plot_testbed_v2_all_empty() # CRITICAL FIX: This line is removed to prevent the crash.
        
        fdtd_params_for_scenario = results['fdtd']['params'] 
        targets_for_scenario = results['fdtd']['targets']
        layers_for_scenario = results['fdtd']['layers']
        tx_pos_x_for_scenario = results['fdtd']['tx_pos_x']
        rx_pos_x_for_scenario = results['fdtd']['rx_pos_x']
        skin_surface_y_plot_for_scenario = results['fdtd']['skin_surface_y_plot']
        
        self.plot_testbed_v2_scenario(targets_for_scenario, layers_for_scenario, 
                                      tx_pos_x_for_scenario, rx_pos_x_for_scenario, 
                                      skin_surface_y_plot_for_scenario, fdtd_params_for_scenario)

        if 'analytical' in results and self.run_fdtd_with_analytical_var.get():
            analytical_data = results['analytical']
            self.plot_testbed_v2_das_2d_analytical(
                analytical_data['image_x_grid'], analytical_data['image_y_grid'], 
                analytical_data['das_image_2d'], analytical_data['targets'], 
                analytical_data['skin_surface_y_plot']
            )
            self.plot_testbed_v2_angular_analytical(
                analytical_data['angles_deg_1d'], analytical_data['das_spectrum_1d'], 
                analytical_data['music_spectrum_1d'], analytical_data['capon_spectrum_1d'], 
                analytical_data['true_angle_deg_1d']
            )
        else:
            self.ax_v2_das_2d_analytical.clear()
            self.ax_v2_das_2d_analytical.set_title("Analytical DAS Image (Not Run)")
            self.ax_v2_angular_analytical.clear()
            self.ax_v2_angular_analytical.set_title("Analytical Angular Spectrum (Not Run)")


        if 'fdtd' in results:
            fdtd_data = results['fdtd']
            self.plot_testbed_v2_das_2d_fdtd(
                fdtd_data['image_x_grid'], fdtd_data['image_y_grid'], 
                fdtd_data['das_image_2d'], fdtd_data['targets'], 
                fdtd_data['skin_surface_y_plot']
            )
            self.plot_testbed_v2_angular_fdtd(
                fdtd_data['angles_deg_1d'], fdtd_data['das_spectrum_1d'], 
                fdtd_data['music_spectrum_1d'], fdtd_data['capon_spectrum_1d'], 
                fdtd_data['true_angle_deg_1d']
            )
        else:
            self.ax_v2_das_2d_fdtd.clear()
            self.ax_v2_das_2d_fdtd.set_title("FDTD DAS Image (Not Run)")
            self.ax_v2_angular_fdtd.clear()
            self.ax_v2_angular_fdtd.set_title("FDTD Angular Spectrum (Not Run)")


    def plot_testbed_v2_all_empty(self):
        self.ax_v2_scenario.clear(); self.ax_v2_scenario.set_title("Testbed Scenario (Common for Analytical & FDTD)"); self.ax_v2_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_v2_scenario.set_aspect('equal', adjustable='box')
        self.ax_v2_das_2d_analytical.clear(); self.ax_v2_das_2d_analytical.set_title("Analytical DAS Image (Empty)"); self.ax_v2_das_2d_analytical.grid(False); self.ax_v2_das_2d_analytical.set_aspect('equal', adjustable='box')
        self.ax_v2_das_2d_fdtd.clear(); self.ax_v2_das_2d_fdtd.set_title("FDTD DAS Image (Empty)"); self.ax_v2_das_2d_fdtd.grid(False); self.ax_v2_das_2d_fdtd.set_aspect('equal', adjustable='box')
        self.ax_v2_angular_analytical.clear(); self.ax_v2_angular_analytical.set_title("Analytical Angular Spectrum (Empty)"); self.ax_v2_angular_analytical.grid(True, linestyle='--', alpha=0.6)
        self.ax_v2_angular_fdtd.clear(); self.ax_v2_angular_fdtd.set_title("FDTD Angular Spectrum (Empty)"); self.ax_v2_angular_fdtd.grid(True, linestyle='--', alpha=0.6)
        self.fig_v2_test_plots.tight_layout(pad=3.0, h_pad=5.0)
        self.canvas_v2_test.draw()

    def plot_testbed_v2_scenario(self, targets, layers, tx_pos_x, rx_pos_x, skin_surface_y_plot, p_v2):
        self.ax_v2_scenario.clear()
        
        array_max_x = max(tx_pos_x[-1] if tx_pos_x.size > 0 else 0, rx_pos_x[-1] if rx_pos_x.size > 0 else 0)
        array_min_x = min(tx_pos_x[0] if tx_pos_x.size > 0 else 0, rx_pos_x[0] if rx_pos_x.size > 0 else 0)
        max_array_width = array_max_x - array_min_x
        plot_width = max_array_width + 20 
        
        current_y_plot = skin_surface_y_plot
        if layers:
            for i, layer in enumerate(layers):
                self.ax_v2_scenario.add_patch(plt.Rectangle((-plot_width/2, current_y_plot - layer['thickness']), 
                                                         plot_width, layer['thickness'], 
                                                         edgecolor='black', facecolor=f'C{i}', alpha=0.5, 
                                                         label=f'Layer {i+1}'))
        
        if targets:
            for i, target in enumerate(targets):
                self.ax_v2_scenario.add_patch(plt.Circle((target['x'], target['y']), target['radius'], 
                                                          color='red', alpha=0.7, label=f'Target {i+1}'))
        
        if tx_pos_x.size > 0:
            self.ax_v2_scenario.plot(tx_pos_x, np.full(len(tx_pos_x), Y_ANTENNA_PLANE), 'gv', markersize=8, linestyle='None', label='Tx')
        if rx_pos_x.size > 0:
            self.ax_v2_scenario.plot(rx_pos_x, np.full(len(rx_pos_x), Y_ANTENNA_PLANE), 'kv', markersize=6, linestyle='None', label='Rx')
        
        self.ax_v2_scenario.set_title("Testbed Scenario (Common for Analytical & FDTD)"); self.ax_v2_scenario.set_xlabel("X (mm)"); self.ax_v2_scenario.set_ylabel("Y (mm)")
        self.ax_v2_scenario.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(targets)+len(layers)+2)
        self.ax_v2_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_v2_scenario.set_aspect('equal', adjustable='box')
        
        min_target_y = min([t['y'] - t['radius'] for t in targets]) if targets else Y_ANTENNA_PLANE 
        min_layer_y = current_y_plot if layers else Y_ANTENNA_PLANE
        min_y_bound = min(min_target_y, min_layer_y) - 5 
        max_y_bound = Y_ANTENNA_PLANE + 5 
        self.ax_v2_scenario.set_ylim(min_y_bound, max_y_bound)
        self.ax_v2_scenario.set_xlim(-plot_width/2, plot_width/2)

    def plot_testbed_v2_das_2d_analytical(self, image_x_grid, image_y_grid, das_image_2d, targets, skin_surface_y_plot):
        if self.cbar_v2_das_analytical is not None:
            self.cbar_v2_das_analytical.remove() 
            self.cbar_v2_das_analytical = None

        self.ax_v2_das_2d_analytical.clear()
        
        image_db = 10 * np.log10(das_image_2d / (np.max(das_image_2d) + 1e-20) + 1e-20)
        
        im = self.ax_v2_das_2d_analytical.imshow(image_db.T, cmap='plasma', 
                                         extent=[image_x_grid[0], image_x_grid[-1], image_y_grid[-1], image_y_grid[0]], 
                                         aspect='auto', interpolation='bilinear', vmin=-30, vmax=0)
        
        if targets:
            for target in targets:
                self.ax_v2_das_2d_analytical.scatter(target['x'], target['y'], s=150, facecolors='none', edgecolors='lime', marker='o', linewidth=2, label=f'GT Target')
        
        self.ax_v2_das_2d_analytical.axhline(y=skin_surface_y_plot, color='cyan', linestyle='--', label='Skin Surface')

        self.ax_v2_das_2d_analytical.set_title("Analytical DAS Image"); self.ax_v2_das_2d_analytical.set_xlabel("X (mm)"); self.ax_v2_das_2d_analytical.set_ylabel("Y (mm)")
        self.ax_v2_das_2d_analytical.legend(bbox_to_anchor=(1.05, 1), loc='upper left');
        self.cbar_v2_das_analytical = self.fig_v2_test_plots.colorbar(im, ax=self.ax_v2_das_2d_analytical, label='Reflectivity (dB)', pad=0.01)
        self.ax_v2_das_2d_analytical.set_aspect('equal', adjustable='box')

    def plot_testbed_v2_das_2d_fdtd(self, image_x_grid, image_y_grid, das_image_2d, targets, skin_surface_y_plot):
        if self.cbar_v2_das_fdtd is not None:
            self.cbar_v2_das_fdtd.remove() 
            self.cbar_v2_das_fdtd = None

        self.ax_v2_das_2d_fdtd.clear()
        
        image_db = 10 * np.log10(das_image_2d / (np.max(das_image_2d) + 1e-20) + 1e-20)
        
        im = self.ax_v2_das_2d_fdtd.imshow(image_db.T, cmap='plasma', 
                                         extent=[image_x_grid[0], image_x_grid[-1], image_y_grid[-1], image_y_grid[0]], 
                                         aspect='auto', interpolation='bilinear', vmin=-30, vmax=0)
        
        if targets:
            for target in targets:
                self.ax_v2_das_2d_fdtd.scatter(target['x'], target['y'], s=150, facecolors='none', edgecolors='lime', marker='o', linewidth=2, label=f'GT Target')
        
        self.ax_v2_das_2d_fdtd.axhline(y=skin_surface_y_plot, color='cyan', linestyle='--', label='Skin Surface')

        self.ax_v2_das_2d_fdtd.set_title("FDTD DAS Image"); self.ax_v2_das_2d_fdtd.set_xlabel("X (mm)"); self.ax_v2_das_2d_fdtd.set_ylabel("Y (mm)")
        self.ax_v2_das_2d_fdtd.legend(bbox_to_anchor=(1.05, 1), loc='upper left');
        self.cbar_v2_das_fdtd = self.fig_v2_test_plots.colorbar(im, ax=self.ax_v2_das_2d_fdtd, label='Reflectivity (dB)', pad=0.01)
        self.ax_v2_das_2d_fdtd.set_aspect('equal', adjustable='box')

    def plot_testbed_v2_angular_analytical(self, angles_deg, das_spectrum, music_spectrum, capon_spectrum, true_angle_deg):
        self.ax_v2_angular_analytical.clear()
        
        self.ax_v2_angular_analytical.plot(angles_deg, das_spectrum, label='DAS')
        self.ax_v2_angular_analytical.plot(angles_deg, music_spectrum, label='MUSIC', linestyle='--')
        self.ax_v2_angular_analytical.plot(angles_deg, capon_spectrum, label='Capon', linestyle=':')
        
        self.ax_v2_angular_analytical.axvline(x=true_angle_deg, color='r', linestyle=':', label=f'True Angle ({true_angle_deg:.1f}°)')
        self.ax_v2_angular_analytical.set_title("Analytical Angular Spectrum"); self.ax_v2_angular_analytical.grid(True, linestyle='--', alpha=0.6); self.ax_v2_angular_analytical.set_xlabel("Angle (Degrees)")
        self.ax_v2_angular_analytical.set_ylabel("Normalized Power"); 
        self.ax_v2_angular_analytical.legend(bbox_to_anchor=(1.05, 1), loc='upper left'); self.ax_v2_angular_analytical.set_ylim(0, 1.1)

    def plot_testbed_v2_angular_fdtd(self, angles_deg, das_spectrum, music_spectrum, capon_spectrum, true_angle_deg):
        self.ax_v2_angular_fdtd.clear()
        
        self.ax_v2_angular_fdtd.plot(angles_deg, das_spectrum, label='DAS')
        self.ax_v2_angular_fdtd.plot(angles_deg, music_spectrum, label='MUSIC', linestyle='--')
        self.ax_v2_angular_fdtd.plot(angles_deg, capon_spectrum, label='Capon', linestyle=':')
        
        self.ax_v2_angular_fdtd.axvline(x=true_angle_deg, color='r', linestyle=':', label=f'True Angle ({true_angle_deg:.1f}°)')
        self.ax_v2_angular_fdtd.set_title("FDTD Angular Spectrum"); self.ax_v2_angular_fdtd.grid(True, linestyle='--', alpha=0.6); self.ax_v2_angular_fdtd.set_xlabel("Angle (Degrees)")
        self.ax_v2_angular_fdtd.set_ylabel("Normalized Power"); 
        self.ax_v2_angular_fdtd.legend(bbox_to_anchor=(1.05, 1), loc='upper left'); self.ax_v2_angular_fdtd.set_ylim(0, 1.1)

    def create_depth_analysis_tab(self):
        main_frame = ttk.Frame(self.tab5, padding="10", style='App.TFrame')
        main_frame.pack(fill=tk.BOTH, expand=True)

        control_outer_frame, control_frame = self._create_scrollable_param_frame(main_frame, text_label="Depth Analysis Parameters")
        control_outer_frame.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)
        
        plot_frame = ttk.Frame(main_frame, padding="10", style='App.TFrame')
        plot_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        self.da_params = {}

        ttk.Label(control_frame, text="--- MIMO Antenna Array ---", style='Header.TLabel').pack(pady=5, anchor='w')
        self.add_param(control_frame, self.da_params, "da_num_tx", "Num Transmitters:", 1)
        self.add_param(control_frame, self.da_params, "da_num_rx", "Num Receivers:", 1)
        
        self.da_min_freq_var = self.add_param(control_frame, self.da_params, "da_min_freq", "Min Frequency (GHz):", 70.0, return_var=True)
        self.da_max_freq_var = self.add_param(control_frame, self.da_params, "da_max_freq", "Max Frequency (GHz):", 80.0, return_var=True)

        self.da_center_freq_display_var = tk.DoubleVar(value=(self.da_min_freq_var.get() + self.da_max_freq_var.get()) / 2)
        self.add_param(control_frame, self.da_params, "da_center_freq_display", "Calc. Center Freq (GHz):", self.da_center_freq_display_var.get(), return_var=True, readonly=True)

        self.da_bandwidth_display_var = tk.DoubleVar(value=self.da_max_freq_var.get() - self.da_min_freq_var.get())
        self.add_param(control_frame, self.da_params, "da_bandwidth_display", "Calc. Bandwidth (GHz):", self.da_bandwidth_display_var.get(), return_var=True, readonly=True)

        self.da_min_freq_var.trace("w", self.update_depth_analysis_derived_freq_params)
        self.da_max_freq_var.trace("w", self.update_depth_analysis_derived_freq_params)
        
        spacing_frame_da = ttk.Frame(control_frame); spacing_frame_da.pack(fill=tk.X, pady=2)
        ttk.Checkbutton(spacing_frame_da, text="Auto-calculate spacing (λ/2)", variable=self.da_auto_spacing_var, command=self.toggle_depth_analysis_spacing_entry).pack(anchor='w', padx=5)
        returned_widgets_da = self.add_param(control_frame, self.da_params, "da_spacing", "Antenna Spacing (mm):", 1.95, return_var_and_widget=True)
        self.da_spacing_entry = returned_widgets_da['widget']

        self.da_antenna_offset_x_entry_widget = self.add_param(control_frame, self.da_params, "da_antenna_offset_x", "Rx X Offset (mm):", 0.0)

        ttk.Label(control_frame, text="--- Medium Properties ---", style='Header.TLabel').pack(pady=10, anchor='w')
        ttk.Checkbutton(control_frame, text="Enable Skin Layers", variable=self.da_layers_enabled_var, command=self.toggle_depth_analysis_layer_entries).pack(anchor='w', padx=5)
        
        da_skin_offset_info = self.add_param(control_frame, self.da_params, "da_skin_surface_offset", "Skin Surface Offset (mm):", 2.5, return_var_and_widget=True) 
        self.da_skin_offset_entry_widget = da_skin_offset_info['widget']
        
        self.da_layer_entries = []
        returned_widget_num_layers_da = self.add_param(control_frame, self.da_params, "da_num_layers", "Num Layers:", 2, return_var_and_widget=True)
        da_num_layers_var = returned_widget_num_layers_da['var']
        self.da_num_layers_entry_widget = returned_widget_num_layers_da['widget']

        outer_layer_scroll_frame_da, self.da_layer_params_frame = self._create_scrollable_param_frame(control_frame, text_label="Skin Layers Properties", min_canvas_height=200)
        outer_layer_scroll_frame_da.pack(fill=tk.X, expand=True, pady=5, padx=5)
        self._create_dynamic_layer_widgets(self.da_layer_params_frame, da_num_layers_var, self.da_layer_entries)
        self.toggle_depth_analysis_layer_entries()

        ttk.Label(control_frame, text="--- Target Properties ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(control_frame, self.da_params, "da_target_x", "Target X Pos (mm):", 0.0)
        self.add_param(control_frame, self.da_params, "da_target_radius", "Target Radius (mm):", 0.75)
        self.add_param(control_frame, self.da_params, "da_target_rcs_sqm", "Target RCS (m²):", 0.001) 

        ttk.Label(control_frame, text="--- System & Detection ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(control_frame, self.da_params, "da_tx_power_dbm", "Tx Power (dBm):", 10.0)
        self.add_param(control_frame, self.da_params, "da_rx_gain_db", "Rx Gain (dB):", 20.0)
        self.add_param(control_frame, self.da_params, "da_noise_figure_db", "Noise Figure (dB):", 3.0)
        self.add_param(control_frame, self.da_params, "da_system_temp_k", "System Temp (K):", 290.0)
        self.add_param(control_frame, self.da_params, "da_min_snr_db", "Min SNR for Detection (dB):", 6.0)

        ttk.Label(control_frame, text="--- Depth Scan Range (Absolute Depth) ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(control_frame, self.da_params, "da_depth_start", "Start Depth (mm):", 5.0) 
        self.add_param(control_frame, self.da_params, "da_depth_end", "End Depth (mm):", 50.0) 
        self.add_param(control_frame, self.da_params, "da_depth_step", "Depth Step (mm):", 0.5) 
                                                                                             
        self.run_da_button = ttk.Button(control_frame, text="Analyze Detection Depth", command=self.run_depth_analysis)
        self.run_da_button.pack(pady=10, fill=tk.X, padx=5)
        self.da_progress_bar = ttk.Progressbar(control_frame, mode='determinate'); self.da_progress_bar.pack(pady=5, fill=tk.X, padx=5)
        self.da_status_label = ttk.Label(control_frame, text="Status: Idle"); self.da_status_label.pack(pady=5, fill=tk.X, padx=5)

        self.fig_da_plots = plt.figure(figsize=(10, 10))
        gs_da = self.fig_da_plots.add_gridspec(3, 1, height_ratios=[1.2, 1, 1], hspace=0.6)

        self.ax_da_scenario = self.fig_da_plots.add_subplot(gs_da[0, 0])
        self.ax_da_snr_vs_depth = self.fig_da_plots.add_subplot(gs_da[1, 0])
        self.ax_da_path_loss_vs_depth = self.fig_da_plots.add_subplot(gs_da[2, 0])
        
        self.canvas_da = FigureCanvasTkAgg(self.fig_da_plots, master=plot_frame); self.canvas_da.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        self.update_depth_analysis_derived_freq_params() 
        self.plot_depth_analysis_all_empty()

    def toggle_depth_analysis_spacing_entry(self):
        self.da_spacing_entry.config(state='disabled' if self.da_auto_spacing_var.get() else 'normal')
        self.update_depth_analysis_spacing()

    def update_depth_analysis_spacing(self, *args):
        if self.da_auto_spacing_var.get():
            try:
                freq_ghz = self.da_center_freq_display_var.get()
                if freq_ghz > 0:
                    lambda_mm = (c / (freq_ghz * 1e9)) * 1000
                    self.da_params['da_spacing'].set(round(lambda_mm / 2, 4))
            except (tk.TclError, KeyError): pass

    def update_depth_analysis_derived_freq_params(self, *args):
        try:
            min_freq = self.da_min_freq_var.get()
            max_freq = self.da_max_freq_var.get()

            if min_freq <= 0 or max_freq <= 0 or min_freq >= max_freq:
                self.da_center_freq_display_var.set(0.0)
                self.da_bandwidth_display_var.set(0.0)
                return

            center_freq = (min_freq + max_freq) / 2
            bandwidth = max_freq - min_freq

            self.da_center_freq_display_var.set(round(center_freq, 4))
            self.da_bandwidth_display_var.set(round(bandwidth, 4))
            
            self.update_depth_analysis_spacing()

        except (tk.TclError, ValueError):
            self.da_center_freq_display_var.set(0.0)
            self.da_bandwidth_display_var.set(0.0)

    def toggle_depth_analysis_layer_entries(self):
        state = 'normal' if self.da_layers_enabled_var.get() else 'disabled'
        if self.da_skin_offset_entry_widget:
            self.da_skin_offset_entry_widget.config(state=state)
        self._toggle_dynamic_widget_frame(self.da_layers_enabled_var, self.da_num_layers_entry_widget, self.da_layer_params_frame)

    def get_depth_analysis_layer_params(self, entries_list):
        return self.get_layer_params(entries_list)

    def run_depth_analysis(self):
        try:
            if self.da_thread and self.da_thread.is_alive():
                messagebox.showwarning("Analysis Running", "A Depth Analysis is already in progress.")
                return

            try:
                p_check = {name: var.get() for name, var in self.da_params.items()}
                
                center_freq_ghz = self.da_center_freq_display_var.get()
                bandwidth_ghz = self.da_bandwidth_display_var.get()
                
                if center_freq_ghz <= 0 or bandwidth_ghz <= 0 or \
                   p_check['da_num_rx'] <= 0 or p_check['da_num_tx'] <= 0 or \
                   p_check['da_target_radius'] <= 0 or p_check['da_target_rcs_sqm'] <= 0 or \
                   p_check['da_system_temp_k'] <= 0 or \
                   p_check['da_depth_start'] <= 0 or p_check['da_depth_end'] <= 0 : 
                    messagebox.showerror("Input Error", "All numeric parameters (frequency range, target radius, RCS, bandwidth, temp, absolute depths) must be positive and valid.")
                    return
                
                if np.isclose(p_check['da_depth_step'], 0.0):
                    messagebox.showerror("Input Error", "Depth Step cannot be zero.")
                    return

                if self.da_layers_enabled_var.get() and p_check['da_skin_surface_offset'] <= 0:
                     messagebox.showerror("Input Error", "Skin Surface Offset must be positive if layers are enabled.")
                     return

                if not (p_check['da_depth_start'] < p_check['da_depth_end'] and p_check['da_depth_step'] > 0):
                    messagebox.showerror("Input Error", "For absolute depths, 'Start Depth' must be less than 'End Depth' for a positive step.")
                    return

            except (tk.TclError, ValueError) as e:
                messagebox.showerror("Input Error", f"Invalid numerical input provided: {e}")
                return

            self.run_da_button.config(state='disabled')
            self.da_results = None
            self.da_status_label.config(text="Status: Initializing Depth Analysis...")
            
            if self.da_progress_bar:
                self.da_progress_bar['value'] = 0
                self.da_progress_bar['maximum'] = 100 
                self.da_progress_bar['mode'] = 'determinate'

            self.da_thread = threading.Thread(target=self._execute_depth_analysis_thread)
            self.da_thread.daemon = True 
            self.da_thread.start()
            self.after(100, self._check_depth_analysis_status)

        except Exception as e:
            error_msg = f"An unexpected error occurred in run_depth_analysis: {e}"
            messagebox.showerror("Critical Error", error_msg)
            self.run_da_button.config(state='normal')
            self.da_status_label.config(text="Status: Critical Error.")
    
    def _execute_depth_analysis_thread(self):
        try:
            p = {name: var.get() for name, var in self.da_params.items()}
            p['layers_enabled'] = self.da_layers_enabled_var.get()

            f_ghz = self.da_center_freq_display_var.get() 
            da_antenna_offset_x = p['da_antenna_offset_x'] 
            
            tx_power_dbm = p['da_tx_power_dbm']
            rx_gain_db = p['da_rx_gain_db']
            noise_figure_db = p['da_noise_figure_db']
            bandwidth_ghz = self.da_bandwidth_display_var.get()
            bandwidth_hz = bandwidth_ghz * 1e9 
            
            system_temp_k = p['da_system_temp_k']
            min_snr_db = p['da_min_snr_db']
            target_rcs_sqm = p['da_target_rcs_sqm']

            depth_start_abs = p['da_depth_start'] 
            depth_end_abs = p['da_depth_end']     
            depth_step_abs = p['da_depth_step']   

            tx_antenna_pos = np.array([0.0, Y_ANTENNA_PLANE])
            rx_antenna_pos = np.array([da_antenna_offset_x, Y_ANTENNA_PLANE])
            
            fc = f_ghz * 1e9 
            lambda_m = c / fc 
            
            layers = self.get_depth_analysis_layer_params(self.da_layer_entries) if p['layers_enabled'] else []
            skin_surface_offset = p['da_skin_surface_offset']
            skin_surface_y_plot = Y_ANTENNA_PLANE - skin_surface_offset

            depth_profile_abs = np.arange(depth_start_abs, depth_end_abs + (depth_step_abs/2), depth_step_abs)
            
            snr_values = np.zeros_like(depth_profile_abs, dtype=float)
            path_loss_values_db = np.zeros_like(depth_profile_abs, dtype=float)
            
            tx_power_W = 10**((tx_power_dbm - 30) / 10)
            rx_gain_linear = 10**(rx_gain_db / 10)
            noise_figure_linear = 10**(noise_figure_db / 10)
            noise_power_W = k * system_temp_k * bandwidth_hz * noise_figure_linear 

            max_detectable_depth_abs = None 
            self.da_status_label.config(text="Status: Calculating SNR Profile...")
            if self.da_progress_bar: self.da_progress_bar['value'] = 10

            num_steps = len(depth_profile_abs)
            for i, current_abs_depth in enumerate(depth_profile_abs):
                target_y_coord = Y_ANTENNA_PLANE - current_abs_depth 
                target_x_for_path = p['da_target_x'] 
                target_pos = np.array([target_x_for_path, target_y_coord])

                total_path_length_tx_target_mm, _, total_attenuation_tx_target_db = \
                    self._calculate_layered_path_metrics(
                        tx_antenna_pos, target_pos, layers, f_ghz, skin_surface_y_plot
                    )
                
                total_path_length_target_rx_mm, _, total_attenuation_target_rx_db = \
                    self._calculate_layered_path_metrics(
                        target_pos, rx_antenna_pos, layers, f_ghz, skin_surface_y_plot
                    )
                
                r1_m = total_path_length_tx_target_mm * 1e-3
                r2_m = total_path_length_target_rx_mm * 1e-3
                
                r1_m = max(r1_m, 1e-6) 
                r2_m = max(r2_m, 1e-6)

                fspl_tx_target_dB = 20 * np.log10(4 * np.pi * r1_m / lambda_m)
                fspl_target_rx_dB = 20 * np.log10(4 * np.pi * r2_m / lambda_m)
                
                total_two_way_path_loss_db = (fspl_tx_target_dB + fspl_target_rx_dB) \
                                             + (total_attenuation_tx_target_db + total_attenuation_target_rx_db)
                
                Pr_dBm = tx_power_dbm + rx_gain_db \
                         + 10 * np.log10(target_rcs_sqm) \
                         - total_two_way_path_loss_db 

                Pn_dBm = 10 * np.log10(noise_power_W) + 30 
                
                snr_db = Pr_dBm - Pn_dBm
                snr_values[i] = snr_db
                path_loss_values_db[i] = total_two_way_path_loss_db

                if snr_db >= min_snr_db:
                    max_detectable_depth_abs = current_abs_depth 
                
                if self.da_progress_bar: 
                    self.da_progress_bar['value'] = 10 + (i / num_steps) * 80 

            self.da_results = {
                "params": p,
                "layers": layers,
                "skin_surface_y_plot": skin_surface_y_plot,
                "tx_antenna_pos": tx_antenna_pos,
                "rx_antenna_pos": rx_antenna_pos,
                "target_x": p['da_target_x'],
                "target_radius": p['da_target_radius'],
                "depth_profile": depth_profile_abs, 
                "snr_values": snr_values,
                "path_loss_values_db": path_loss_values_db,
                "min_snr_db": min_snr_db,
                "max_detectable_depth": max_detectable_depth_abs 
            }
            if self.da_progress_bar: self.da_progress_bar['value'] = 100
        except Exception as e:
            import traceback
            error_msg = f"{e}\n{traceback.format_exc()}"
            self.da_results = {"error": error_msg}

    def _check_depth_analysis_status(self):
        if self.da_results:
            self.run_da_button.config(state='normal')
            if self.da_progress_bar: self.da_progress_bar.stop()
            if "error" in self.da_results: 
                messagebox.showerror("Depth Analysis Error", str(self.da_results["error"]))
                self.da_status_label.config(text="Status: Error occurred.")
            else:
                data = self.da_results
                max_detectable_y_coord = Y_ANTENNA_PLANE - data['max_detectable_depth'] if data['max_detectable_depth'] is not None else None
                
                self.plot_da_scenario(data['tx_antenna_pos'], data['rx_antenna_pos'], data['layers'], 
                                      data['skin_surface_y_plot'], data['target_x'], data['target_radius'], 
                                      max_detectable_y_coord) 

                self.plot_snr_vs_depth(data['depth_profile'], data['snr_values'], data['min_snr_db'], data['max_detectable_depth'])
                self.plot_path_loss_vs_depth(data['depth_profile'], data['path_loss_values_db'])
                
                self.canvas_da.draw()
                self.da_status_label.config(text=f"Status: Analysis Complete. Max Detectable Depth: {data['max_detectable_depth']:.2f} mm" if data['max_detectable_depth'] is not None else "Status: Analysis Complete. Target not detected.")
            
            self.da_thread = None 
        else: 
            self.da_status_label.config(text="Status: Running...")
            self.after(100, self._check_depth_analysis_status)

    def plot_depth_analysis_all_empty(self):
        self.ax_da_scenario.clear(); self.ax_da_scenario.set_title("Depth Analysis Scenario (Empty)"); self.ax_da_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_da_scenario.set_aspect('equal', adjustable='box')
        self.ax_da_snr_vs_depth.clear(); self.ax_da_snr_vs_depth.set_title("SNR vs. Depth (Empty)"); self.ax_da_snr_vs_depth.set_xlabel("Absolute Depth (mm)"); self.ax_da_snr_vs_depth.set_ylabel("SNR (dB)"); self.ax_da_snr_vs_depth.grid(True, linestyle='--', alpha=0.6)
        self.ax_da_path_loss_vs_depth.clear(); self.ax_da_path_loss_vs_depth.set_title("Total Path Loss vs. Depth (Two-Way, Empty)"); self.ax_da_path_loss_vs_depth.set_xlabel("Absolute Depth (mm)"); self.ax_da_path_loss_vs_depth.set_ylabel("Path Loss (dB)"); self.ax_da_path_loss_vs_depth.grid(True, linestyle='--', alpha=0.6)
        self.fig_da_plots.tight_layout(pad=3.0)
        self.canvas_da.draw()

    def plot_da_scenario(self, tx_pos, rx_pos, layers, skin_surface_y_plot, target_x, target_radius, max_detectable_y_coord):
        self.ax_da_scenario.clear()
        
        num_tx = int(self.da_params['da_num_tx'].get())
        num_rx = int(self.da_params['da_num_rx'].get())
        spacing = self.da_params['da_spacing'].get()
        
        all_x_coords = [tx_pos[0], rx_pos[0], target_x]
        
        if num_tx > 1:
            tx_array_width = (num_tx - 1) * spacing
            all_x_coords.extend([tx_pos[0] - (tx_array_width/2), tx_pos[0] + (tx_array_width/2)])
        if num_rx > 1:
            rx_array_width = (num_rx - 1) * spacing
            rx_offset_x = self.da_params['da_antenna_offset_x'].get()
            all_x_coords.extend([rx_pos[0] - (rx_array_width/2), rx_pos[0] + (rx_array_width/2)])

        max_abs_x = max(abs(x) for x in all_x_coords)
        plot_width_x = max_abs_x * 2 + 10 

        current_y_plot = skin_surface_y_plot
        if layers:
            for i, layer in enumerate(layers):
                self.ax_da_scenario.add_patch(plt.Rectangle((-plot_width_x/2, current_y_plot - layer['thickness']), 
                                                         plot_width_x, layer['thickness'], 
                                                         edgecolor='black', facecolor=f'C{i}', alpha=0.5, 
                                                         label=f'Layer {i+1}'))
        
        tx_positions_x = np.arange(num_tx) * spacing - (num_tx - 1) * spacing / 2
        self.ax_da_scenario.plot(tx_positions_x, np.full(num_tx, Y_ANTENNA_PLANE), 'gv', markersize=8, linestyle='None', label='Tx')
        
        rx_positions_x = np.arange(num_rx) * spacing - (num_rx - 1) * spacing / 2 + self.da_params['da_antenna_offset_x'].get()
        self.ax_da_scenario.plot(rx_positions_x, np.full(num_rx, Y_ANTENNA_PLANE), 'kv', markersize=6, linestyle='None', label='Rx')
        
        if max_detectable_y_coord is not None:
            self.ax_da_scenario.add_patch(plt.Circle((target_x, max_detectable_y_coord), target_radius, 
                                                      color='red', alpha=0.7, label=f'Max Depth ({abs(max_detectable_y_coord):.1f} mm)'))
        else:
            shallowest_scan_y_coord = Y_ANTENNA_PLANE - self.da_params['da_depth_start'].get()
            self.ax_da_scenario.add_patch(plt.Circle((target_x, shallowest_scan_y_coord), target_radius, 
                                                      color='gray', alpha=0.3, label=f'Scan Start'))


        self.ax_da_scenario.set_title("Depth Analysis Scenario"); self.ax_da_scenario.set_xlabel("X (mm)"); self.ax_da_scenario.set_ylabel("Y (mm)")
        self.ax_da_scenario.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(layers)+3)
        self.ax_da_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_da_scenario.set_aspect('equal', adjustable='box')
        
        min_y_val_in_plot = Y_ANTENNA_PLANE - self.da_params['da_depth_end'].get() - 5 
        max_y_val_in_plot = Y_ANTENNA_PLANE + 5 
        
        self.ax_da_scenario.set_ylim(min_y_val_in_plot, max_y_val_in_plot)
        self.ax_da_scenario.set_xlim(-plot_width_x/2, plot_width_x/2) 

    def plot_snr_vs_depth(self, depth_profile, snr_values, min_snr_db, max_detectable_depth):
        self.ax_da_snr_vs_depth.clear()
        self.ax_da_snr_vs_depth.plot(depth_profile, snr_values, 'b-o', markersize=4, label='Calculated SNR')
        self.ax_da_snr_vs_depth.axhline(y=min_snr_db, color='r', linestyle='--', label=f'Min Detection SNR')
        
        if max_detectable_depth is not None:
            self.ax_da_snr_vs_depth.axvline(x=max_detectable_depth, color='g', linestyle=':', 
                                            label=f'Max Depth ({max_detectable_depth:.1f} mm)')
        
        self.ax_da_snr_vs_depth.set_title("SNR vs. Depth")
        self.ax_da_snr_vs_depth.set_xlabel("Absolute Depth (mm)"); self.ax_da_snr_vs_depth.set_ylabel("SNR (dB)")
        self.ax_da_snr_vs_depth.legend(bbox_to_anchor=(1.05, 1), loc='upper left');
        self.ax_da_snr_vs_depth.grid(True, linestyle='--', alpha=0.6)
        x_min = min(depth_profile) - abs(self.da_params['da_depth_step'].get())
        x_max = max(depth_profile) + abs(self.da_params['da_depth_step'].get())
        self.ax_da_snr_vs_depth.set_xlim(x_min, x_max)
        y_min = min(np.min(snr_values), min_snr_db) - 5
        y_max = max(np.max(snr_values), min_snr_db) + 5
        self.ax_da_snr_vs_depth.set_ylim(y_min, y_max)


    def plot_path_loss_vs_depth(self, depth_profile, path_loss_values_db):
        self.ax_da_path_loss_vs_depth.clear()
        self.ax_da_path_loss_vs_depth.plot(depth_profile, path_loss_values_db, 'm-o', markersize=4, label='Total 2-Way Path Loss')
        
        self.ax_da_path_loss_vs_depth.set_title("Total Path Loss vs. Depth (Two-Way)")
        self.ax_da_path_loss_vs_depth.set_xlabel("Absolute Depth (mm)"); self.ax_da_path_loss_vs_depth.set_ylabel("Path Loss (dB)")
        self.ax_da_path_loss_vs_depth.legend(bbox_to_anchor=(1.05, 1), loc='upper left');
        self.ax_da_path_loss_vs_depth.grid(True, linestyle='--', alpha=0.6)
        x_min = min(depth_profile) - abs(self.da_params['da_depth_step'].get())
        x_max = max(depth_profile) + abs(self.da_params['da_depth_step'].get())
        self.ax_da_path_loss_vs_depth.set_xlim(x_min, x_max)
        y_min = np.min(path_loss_values_db) - 10
        y_max = np.max(path_loss_values_db) + 10
        self.ax_da_path_loss_vs_depth.set_ylim(y_min, y_max)


    # ===================================================================
    # TAB 6: PARAMETRIC ANALYSIS (NEWLY ADDED)
    # ===================================================================
    def create_parametric_analysis_tab(self):
        main_frame = ttk.Frame(self.tab6, padding="10", style='App.TFrame')
        main_frame.pack(fill=tk.BOTH, expand=True)

        control_outer_frame, control_frame = self._create_scrollable_param_frame(main_frame, text_label="Parametric Analysis Controls")
        control_outer_frame.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)
        
        plot_frame = ttk.Frame(main_frame, padding="10", style='App.TFrame')
        plot_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        self.pa_params = {}

        # --- Analysis Type ---
        ttk.Label(control_frame, text="--- Analysis Configuration ---", style='Header.TLabel').pack(pady=5, anchor='w')
        sim_method_frame = ttk.Frame(control_frame); sim_method_frame.pack(fill=tk.X, pady=2, padx=5)
        ttk.Label(sim_method_frame, text="Simulation Method:", width=22).pack(side=tk.LEFT)
        ttk.OptionMenu(sim_method_frame, self.pa_simulation_method_var, "Analytical", "Analytical", "FDTD").pack(side=tk.LEFT, expand=True, fill=tk.X)

        analysis_var_frame = ttk.Frame(control_frame); analysis_var_frame.pack(fill=tk.X, pady=2, padx=5)
        ttk.Label(analysis_var_frame, text="Analysis Variable:", width=22).pack(side=tk.LEFT)
        analysis_options = ["Frequency (GHz)", "Bandwidth (MHz)", "Tx Power (dBm)", "Target RCS (m²)", "Layer 1 Permittivity", "Layer 1 Conductivity (S/m)"]
        ttk.OptionMenu(analysis_var_frame, self.pa_analysis_variable_var, analysis_options[0], *analysis_options, command=self.update_pa_variable_controls).pack(side=tk.LEFT, expand=True, fill=tk.X)
        
        # --- Sweep Ranges ---
        ttk.Label(control_frame, text="--- Sweep Ranges ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(control_frame, self.pa_params, "pa_depth_min", "Depth Min (mm):", 5.0)
        self.add_param(control_frame, self.pa_params, "pa_depth_max", "Depth Max (mm):", 40.0)
        self.add_param(control_frame, self.pa_params, "pa_depth_steps", "Depth Steps:", 20)

        self.pa_variable_frame = ttk.Frame(control_frame)
        self.pa_variable_frame.pack(fill=tk.X, pady=2, padx=5)
        self.pa_variable_label = ttk.Label(self.pa_variable_frame, text="Var Min:", width=22)
        self.pa_variable_label.pack(side=tk.LEFT)
        self.pa_variable_min_var = tk.DoubleVar(value=60.0)
        self.pa_variable_min_entry = ttk.Entry(self.pa_variable_frame, textvariable=self.pa_variable_min_var)
        self.pa_variable_min_entry.pack(side=tk.LEFT, expand=True, fill=tk.X)

        self.add_param(control_frame, self.pa_params, "pa_variable_max", "Var Max:", 80.0)
        self.add_param(control_frame, self.pa_params, "pa_variable_steps", "Var Steps:", 20)
        
        # --- Fixed Parameters (a subset of other tabs) ---
        ttk.Label(control_frame, text="--- Fixed Parameters ---", style='Header.TLabel').pack(pady=10, anchor='w')
        self.add_param(control_frame, self.pa_params, "pa_freq_ghz", "Center Freq (GHz):", 77.0)
        self.add_param(control_frame, self.pa_params, "pa_bandwidth_mhz", "Bandwidth (MHz):", 200)
        self.add_param(control_frame, self.pa_params, "pa_tx_power_dbm", "Tx Power (dBm):", 10.0)
        self.add_param(control_frame, self.pa_params, "pa_target_rcs_sqm", "Target RCS (m²):", 0.001)
        self.add_param(control_frame, self.pa_params, "pa_target_x", "Target X (mm):", 0.0)
        self.add_param(control_frame, self.pa_params, "pa_min_snr_db", "Min Detection SNR (dB):", 6.0)
        
        # FDTD Specific
        self.add_param(control_frame, self.pa_params, "pa_resolution", "FDTD Resolution:", 10)
        
        # --- Medium ---
        ttk.Label(control_frame, text="--- Medium Properties ---", style='Header.TLabel').pack(pady=10, anchor='w')
        ttk.Checkbutton(control_frame, text="Enable Skin Layers", variable=self.pa_layers_enabled_var, command=self.toggle_pa_layer_entries).pack(anchor='w', padx=5)
        
        pa_skin_offset_info = self.add_param(control_frame, self.pa_params, "pa_skin_surface_offset", "Skin Surface Offset (mm):", 2.5, return_var_and_widget=True) 
        self.pa_skin_offset_entry_widget = pa_skin_offset_info['widget']
        
        self.pa_layer_entries = []
        pa_num_layers_info = self.add_param(control_frame, self.pa_params, "pa_num_layers", "Num Layers:", 2, return_var_and_widget=True)
        pa_num_layers_var = pa_num_layers_info['var']
        self.pa_num_layers_entry_widget = pa_num_layers_info['widget']

        outer_layer_scroll_frame_pa, self.pa_layer_params_frame = self._create_scrollable_param_frame(control_frame, text_label="Skin Layers Properties (Fixed)", min_canvas_height=150)
        outer_layer_scroll_frame_pa.pack(fill=tk.X, expand=True, pady=5, padx=5)
        self._create_dynamic_layer_widgets(self.pa_layer_params_frame, pa_num_layers_var, self.pa_layer_entries)
        
        # --- Run Controls ---
        self.run_pa_button = ttk.Button(control_frame, text="Run Parametric Analysis", command=self.run_parametric_analysis)
        self.run_pa_button.pack(pady=10, fill=tk.X, padx=5)
        self.pa_progress_bar = ttk.Progressbar(control_frame, mode='determinate'); self.pa_progress_bar.pack(pady=5, fill=tk.X, padx=5)
        self.pa_status_label = ttk.Label(control_frame, text="Status: Idle"); self.pa_status_label.pack(pady=5, fill=tk.X, padx=5)

        # --- Initialize Plots ---
        self.fig_pa_plots, (self.ax_pa_scenario, self.ax_pa_heatmap) = plt.subplots(1, 2, figsize=(14, 7), gridspec_kw={'width_ratios': [1, 2]})
        self.canvas_pa = FigureCanvasTkAgg(self.fig_pa_plots, master=plot_frame)
        self.canvas_pa.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        self.update_pa_variable_controls() # Set initial state
        self.toggle_pa_layer_entries()
        self.plot_pa_all_empty()

    def update_pa_variable_controls(self, *args):
        variable_name = self.pa_analysis_variable_var.get()
        if variable_name == "Frequency (GHz)":
            self.pa_variable_label.config(text="Frequency Min (GHz):")
            self.pa_params['pa_variable_max'].set(80.0)
            self.pa_variable_min_var.set(60.0)
        elif variable_name == "Bandwidth (MHz)":
            self.pa_variable_label.config(text="Bandwidth Min (MHz):")
            self.pa_params['pa_variable_max'].set(1000.0)
            self.pa_variable_min_var.set(100.0)
        elif variable_name == "Tx Power (dBm)":
            self.pa_variable_label.config(text="Tx Power Min (dBm):")
            self.pa_params['pa_variable_max'].set(20.0)
            self.pa_variable_min_var.set(0.0)
        elif variable_name == "Target RCS (m²)":
            self.pa_variable_label.config(text="RCS Min (m²):")
            self.pa_params['pa_variable_max'].set(0.01)
            self.pa_variable_min_var.set(0.0001)
        elif variable_name == "Layer 1 Permittivity":
            self.pa_variable_label.config(text="L1 εᵣ Min:")
            self.pa_params['pa_variable_max'].set(60.0)
            self.pa_variable_min_var.set(20.0)
        elif variable_name == "Layer 1 Conductivity (S/m)":
            self.pa_variable_label.config(text="L1 Cond. Min (S/m):")
            self.pa_params['pa_variable_max'].set(2.0)
            self.pa_variable_min_var.set(0.01)
        
        self.pa_params['pa_variable_min'] = self.pa_variable_min_var

    def toggle_pa_layer_entries(self):
        state = 'normal' if self.pa_layers_enabled_var.get() else 'disabled'
        if self.pa_skin_offset_entry_widget:
            self.pa_skin_offset_entry_widget.config(state=state)
        self._toggle_dynamic_widget_frame(self.pa_layers_enabled_var, self.pa_num_layers_entry_widget, self.pa_layer_params_frame)

    def run_parametric_analysis(self):
        if self.pa_thread and self.pa_thread.is_alive():
            messagebox.showwarning("Analysis Running", "A Parametric Analysis is already in progress.")
            return

        sim_method = self.pa_simulation_method_var.get()
        if sim_method == "FDTD":
            steps = self.pa_params['pa_depth_steps'].get() * self.pa_params['pa_variable_steps'].get()
            msg = f"You are about to run a Parametric Analysis in FDTD mode. This will execute {int(steps)} separate FDTD simulations, which could take an extremely long time. Are you sure you want to proceed?"
            if not messagebox.askyesno("FDTD Performance Warning", msg):
                return

        self.run_pa_button.config(state='disabled')
        self.pa_results = None
        self.pa_status_label.config(text="Status: Initializing Analysis...")
        self.pa_progress_bar['value'] = 0

        self.pa_thread = threading.Thread(target=self._execute_parametric_analysis_thread)
        self.pa_thread.daemon = True
        self.pa_thread.start()
        self.after(100, self._check_parametric_analysis_status)

    def _execute_parametric_analysis_thread(self):
        try:
            p = {name: var.get() for name, var in self.pa_params.items()}
            p['layers_enabled'] = self.pa_layers_enabled_var.get()
            p['analysis_variable'] = self.pa_analysis_variable_var.get()
            p['simulation_method'] = self.pa_simulation_method_var.get()
            
            layers = self.get_layer_params(self.pa_layer_entries) if p['layers_enabled'] else []

            depth_range = np.linspace(p['pa_depth_min'], p['pa_depth_max'], int(p['pa_depth_steps']))
            var_range = np.linspace(p['pa_variable_min'], p['pa_variable_max'], int(p['pa_variable_steps']))
            
            snr_heatmap = np.zeros((len(depth_range), len(var_range)))
            
            total_iterations = len(depth_range) * len(var_range)
            current_iteration = 0

            for i, depth in enumerate(depth_range):
                for j, var_value in enumerate(var_range):
                    current_iteration += 1
                    self.pa_status_label.config(text=f"Status: Running {current_iteration}/{total_iterations}...")
                    self.pa_progress_bar['value'] = (current_iteration / total_iterations) * 100

                    # Create a temporary parameter set for this iteration
                    iter_p = p.copy()
                    iter_layers = [layer.copy() for layer in layers] # Deep copy for modification
                    
                    # Update the parameter that is being swept
                    if p['analysis_variable'] == "Frequency (GHz)":
                        iter_p['pa_freq_ghz'] = var_value
                    elif p['analysis_variable'] == "Bandwidth (MHz)":
                        iter_p['pa_bandwidth_mhz'] = var_value
                    elif p['analysis_variable'] == "Tx Power (dBm)":
                        iter_p['pa_tx_power_dbm'] = var_value
                    elif p['analysis_variable'] == "Target RCS (m²)":
                        iter_p['pa_target_rcs_sqm'] = var_value
                    elif p['analysis_variable'] == "Layer 1 Permittivity":
                        if len(iter_layers) > 0: iter_layers[0]['permit'] = var_value
                    elif p['analysis_variable'] == "Layer 1 Conductivity (S/m)":
                        if len(iter_layers) > 0: iter_layers[0]['conductivity'] = var_value

                    # Calculate SNR for this point
                    target_pos = np.array([iter_p['pa_target_x'], Y_ANTENNA_PLANE - depth])
                    
                    snr_db = self._calculate_single_point_snr(iter_p, iter_layers, target_pos)
                    snr_heatmap[i, j] = snr_db

            self.pa_results = {
                "params": p,
                "layers": layers,
                "depth_range": depth_range,
                "var_range": var_range,
                "snr_heatmap": snr_heatmap,
            }

        except Exception as e:
            import traceback
            self.pa_results = {"error": f"{e}\n{traceback.format_exc()}"}
            
    def _calculate_single_point_snr(self, p, layers, target_pos):
        """Calculates SNR for a single configuration point."""
        f_ghz = p['pa_freq_ghz']
        bandwidth_mhz = p['pa_bandwidth_mhz']
        tx_power_dbm = p['pa_tx_power_dbm']
        target_rcs = p['pa_target_rcs_sqm']
        skin_surface_offset = p['pa_skin_surface_offset']
        skin_surface_y_plot = Y_ANTENNA_PLANE - skin_surface_offset
        
        # System noise calculation (common for both methods)
        noise_figure_db = 3.0 # Assuming fixed from other tabs for simplicity
        system_temp_k = 290.0 # Assuming fixed
        bandwidth_hz = bandwidth_mhz * 1e9
        noise_power_W = k * system_temp_k * bandwidth_hz * (10**(noise_figure_db / 10))
        Pn_dBm = 10 * np.log10(noise_power_W) + 30

        # Use Analytical Method
        if p['simulation_method'] == "Analytical":
            tx_antenna_pos = np.array([0.0, Y_ANTENNA_PLANE])
            rx_antenna_pos = np.array([0.0, Y_ANTENNA_PLANE]) # Assuming co-located for this tool
            
            _, _, atten_tx_db = self._calculate_layered_path_metrics(tx_antenna_pos, target_pos, layers, f_ghz, skin_surface_y_plot)
            path_len_tx_m, _, _ = self._calculate_layered_path_metrics(tx_antenna_pos, target_pos, [], f_ghz, skin_surface_y_plot) # Free space path
            path_len_tx_m *= 1e-3

            lambda_m = c / (f_ghz * 1e9)
            fspl_db = 2 * (20 * np.log10(4 * np.pi * path_len_tx_m / lambda_m))
            total_loss = fspl_db + (2 * atten_tx_db)

            Pr_dBm = tx_power_dbm + 20 - total_loss + 10 * np.log10(target_rcs) # 20 is placeholder for gain
            return Pr_dBm - Pn_dBm

        # Use FDTD Method
        elif p['simulation_method'] == "FDTD":
            # This is a simplified FDTD run for signal strength, not a full MIMO sim
            pml = 2.0
            domain_x = 20
            domain_y = abs(target_pos[1]) + skin_surface_offset + 10 + 2*pml
            cell = mp.Vector3(domain_x, domain_y, 0)
            meep_y_antenna = (domain_y/2) - pml
            
            # Healthy geometry
            geometry_h = []
            current_y = meep_y_antenna - skin_surface_offset
            for layer in layers:
                center_y = current_y - layer['thickness'] / 2
                geometry_h.append(mp.Block(center=mp.Vector3(0, center_y), size=mp.Vector3(mp.inf, layer['thickness']), material=mp.Medium(epsilon=layer['permit'], D_conductivity=layer['conductivity']/(2*np.pi*f_ghz*1e9*epsilon_0))))
                current_y -= layer['thickness']
            
            # Unhealthy geometry
            geometry_u = list(geometry_h)
            target_y_meep = meep_y_antenna + target_pos[1]
            geometry_u.append(mp.Cylinder(center=mp.Vector3(target_pos[0], target_y_meep), radius=0.5, material=mp.Medium(epsilon=60)))

            fcen_meep = f_ghz * 1e9 * MEEP_UNIT_LENGTH_M / c
            df_meep = (bandwidth_mhz * 1e6) * MEEP_UNIT_LENGTH_M / c
            
            sources = [mp.Source(src=mp.GaussianSource(frequency=fcen_meep, fwidth=df_meep), component=mp.Ez, center=mp.Vector3(0, meep_y_antenna))]
            
            # Run Healthy
            sim_h = mp.Simulation(cell_size=cell, boundary_layers=[mp.PML(pml)], geometry=geometry_h, sources=sources, resolution=p['pa_resolution'])
            dft_h = sim_h.add_dft_fields([mp.Ez], fcen_meep, df_meep, 1, where=mp.Volume(center=mp.Vector3(0, meep_y_antenna)))
            sim_h.run(until=2 * (domain_y * np.sqrt(60)) ) # Estimate run time
            sig_h = sim_h.get_dft_array(dft_h, mp.Ez, 0)
            sim_h.reset_meep()

            # Run Unhealthy
            sim_u = mp.Simulation(cell_size=cell, boundary_layers=[mp.PML(pml)], geometry=geometry_u, sources=sources, resolution=p['pa_resolution'])
            dft_u = sim_u.add_dft_fields([mp.Ez], fcen_meep, df_meep, 1, where=mp.Volume(center=mp.Vector3(0, meep_y_antenna)))
            sim_u.run(until=2 * (domain_y * np.sqrt(60)) )
            sig_u = sim_u.get_dft_array(dft_u, mp.Ez, 0)
            sim_u.reset_meep()
            
            # Very rough conversion from Meep units to power for SNR
            differential_signal = abs(sig_u - sig_h)
            Pr_W = (differential_signal**2) * 1e-12 # Heuristic scaling factor
            Pr_dBm = 10 * np.log10(Pr_W) + 30
            
            return Pr_dBm - Pn_dBm

        return 0 # Should not happen

    def _check_parametric_analysis_status(self):
        if self.pa_results:
            self.run_pa_button.config(state='normal')
            if "error" in self.pa_results:
                messagebox.showerror("Parametric Analysis Error", self.pa_results['error'])
                self.pa_status_label.config(text="Status: Error!")
            else:
                self.pa_status_label.config(text="Status: Analysis Complete.")
                self.plot_pa_results(self.pa_results)
            self.pa_thread = None
        else:
            self.after(100, self._check_parametric_analysis_status)

    def plot_pa_all_empty(self):
        self.ax_pa_scenario.clear()
        self.ax_pa_scenario.set_title("Scenario Preview")
        self.ax_pa_scenario.set_xlabel("X (mm)")
        self.ax_pa_scenario.set_ylabel("Y (mm)")
        self.ax_pa_scenario.grid(True, linestyle='--', alpha=0.6)
        self.ax_pa_scenario.set_aspect('equal', adjustable='box')

        self.ax_pa_heatmap.clear()
        self.ax_pa_heatmap.set_title("SNR Heatmap")
        self.ax_pa_heatmap.set_xlabel("Parameter Sweep")
        self.ax_pa_heatmap.set_ylabel("Absolute Depth (mm)")
        
        self.fig_pa_plots.tight_layout(pad=3.0)
        self.canvas_pa.draw()

    def plot_pa_results(self, data):
        p = data['params']
        layers = data['layers']
        depth_range = data['depth_range']
        var_range = data['var_range']
        snr_heatmap = data['snr_heatmap']

        # Plot Scenario Preview
        self.ax_pa_scenario.clear()
        plot_width = 30
        current_y_plot = Y_ANTENNA_PLANE - p['pa_skin_surface_offset']
        if layers and p['layers_enabled']:
            for i, layer in enumerate(layers):
                self.ax_pa_scenario.add_patch(plt.Rectangle((-plot_width/2, current_y_plot - layer['thickness']), plot_width, layer['thickness'], edgecolor='black', facecolor=f'C{i}', alpha=0.5, label=f'Layer {i+1}'))
        self.ax_pa_scenario.plot(0, 0, 'gv', markersize=8, label='Tx/Rx')
        self.ax_pa_scenario.add_patch(plt.Circle((p['pa_target_x'], Y_ANTENNA_PLANE - np.mean(depth_range)), 0.5, color='red', alpha=0.7, label='Target'))
        self.ax_pa_scenario.set_title("Scenario Preview"); self.ax_pa_scenario.set_xlabel("X (mm)"); self.ax_pa_scenario.set_ylabel("Y (mm)")
        self.ax_pa_scenario.legend()
        self.ax_pa_scenario.grid(True, linestyle='--', alpha=0.6); self.ax_pa_scenario.set_aspect('equal', adjustable='box')
        self.ax_pa_scenario.set_ylim(Y_ANTENNA_PLANE - p['pa_depth_max'] - 5, Y_ANTENNA_PLANE + 5)
        self.ax_pa_scenario.set_xlim(-plot_width/2, plot_width/2)

        # Plot Heatmap
        if self.cbar_pa_heatmap:
            self.cbar_pa_heatmap.remove()
        self.ax_pa_heatmap.clear()
        im = self.ax_pa_heatmap.imshow(snr_heatmap.T, cmap='viridis', aspect='auto', origin='lower', extent=[depth_range[0], depth_range[-1], var_range[0], var_range[-1]])
        
        # Add contour for detection threshold
        self.ax_pa_heatmap.contour(depth_range, var_range, snr_heatmap.T, levels=[p['pa_min_snr_db']], colors='red', linestyles='--')

        self.ax_pa_heatmap.set_title("SNR Analysis Heatmap")
        self.ax_pa_heatmap.set_xlabel("Absolute Depth (mm)")
        self.ax_pa_heatmap.set_ylabel(p['analysis_variable'])
        self.cbar_pa_heatmap = self.fig_pa_plots.colorbar(im, ax=self.ax_pa_heatmap, label='SNR (dB)')
        
        self.fig_pa_plots.tight_layout(pad=3.0)
        self.canvas_pa.draw()


if __name__ == "__main__":
    app = SkinCancerSimulator()
    app.mainloop()

ULTRA-DEBUG: SkinCancerSimulator __init__ started.
DEBUG: Selected 'sans-serif' as the base GUI font.
ULTRA-DEBUG: create_skin_imaging_tab started.
ULTRA-DEBUG: create_skin_imaging_tab completed.
ULTRA-DEBUG: create_algorithm_testbed_tab started.
ULTRA-DEBUG: create_algorithm_testbed_tab completed.


/tmp/ipykernel_48710/2664065451.py:2165: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  self.fig_v2_test_plots.tight_layout(pad=3.0, h_pad=5.0)
/tmp/ipykernel_48710/2664065451.py:2602: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  self.fig_da_plots.tight_layout(pad=3.0)


ULTRA-DEBUG: SkinCancerSimulator __init__ completed.


-----------
Initializing structure...
time for choose_chunkdivision = 0.00100398 s
Working in 2D dimensions.
Computational cell is 46.7333 x 27 x 0 with resolution 15
     block, center = (0,8.25,0)
          size (46.7209,1.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (38,38,38)
     block, center = (0,6.25,0)
          size (46.7209,2.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (45,45,45)
time for set_epsilon = 0.610794 s
time for set_conductivity = 0.0157979 s
time for set_conductivity = 0.0296981 s
time for set_conductivity = 0.0228002 s
-----------
Meep progress: 36.53333333333333/431.8863824383493 = 8.5% done in 4.0s, 43.3s to go
on time step 1096 (time=36.5333), 0.00365077 s/step
Meep progress: 77.6/431.8863824383493 = 18.0% done in 8.0s, 36.5s to go
on time step 2328 (time=77.6), 0.00324703 s/step
Meep progress: 119.23333333333333/431.8863824383493 = 27.6% done in 12.0s,

-----------
Initializing structure...
time for choose_chunkdivision = 0.000603914 s
Working in 2D dimensions.
Computational cell is 46.7333 x 27 x 0 with resolution 15
     block, center = (0,8.25,0)
          size (46.7209,1.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (38,38,38)
     block, center = (0,6.25,0)
          size (46.7209,2.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (45,45,45)
     cylinder, center = (2,7.5,0)
          radius 0.75, height 1e+20, axis (0, 0, 1)
          dielectric constant epsilon diagonal = (60,60,60)
time for set_epsilon = 0.734574 s
time for set_conductivity = 0.0290489 s
time for set_conductivity = 0.018996 s
time for set_conductivity = 0.024097 s
-----------
Meep progress: 20.8/431.8863824383493 = 4.8% done in 4.0s, 79.1s to go
on time step 624 (time=20.8), 0.00641257 s/step
Meep progress: 42.833333333333336/431.8863824383493 = 9.9% done in 

...

run 0 finished at t = 507.6666666666667 (15230 timesteps)
-----------
Initializing structure...
time for choose_chunkdivision = 0.000525951 s
Working in 2D dimensions.
Computational cell is 46.7333 x 25.2667 x 0 with resolution 15
     block, center = (0,7.375,0)
          size (46.7208,1.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (38,38,38)
     block, center = (0,5.375,0)
          size (46.7208,2.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (45,45,45)
     cylinder, center = (0,6.625,0)
          radius 0.75, height 1e+20, axis (0, 0, 1)
          dielectric constant epsilon diagonal = (60,60,60)
time for set_epsilon = 0.541956 s
time for set_conductivity = 0.014991 s
time for set_conductivity = 0.015136 s
time for set_conductivity = 0.014524 s
-----------
Meep progress: 45.266666666666666/507.6456362796297 = 8.9% done in 4.0s, 40.9s to go
on time step 1358 (time=45.2667), 0

run 0 finished at t = 507.6666666666667 (15230 timesteps)
-----------
Initializing structure...
time for choose_chunkdivision = 0.00046277 s
Working in 2D dimensions.
Computational cell is 46.7333 x 25.2667 x 0 with resolution 15
     block, center = (0,7.375,0)
          size (46.7208,1.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (38,38,38)
     block, center = (0,5.375,0)
          size (46.7208,2.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (45,45,45)
time for set_epsilon = 0.847724 s
time for set_conductivity = 0.0145741 s
time for set_conductivity = 0.027818 s
time for set_conductivity = 0.024075 s
-----------
Meep progress: 39.7/507.6456362796297 = 7.8% done in 4.0s, 47.2s to go
on time step 1191 (time=39.7), 0.00336133 s/step
Meep progress: 81.16666666666667/507.6456362796297 = 16.0% done in 8.0s, 42.1s to go
on time step 2435 (time=81.1667), 0.00321642 s/step
Meep progre

run 0 finished at t = 507.6666666666667 (15230 timesteps)
-----------
Initializing structure...
time for choose_chunkdivision = 0.00068593 s
Working in 2D dimensions.
Computational cell is 46.7333 x 25.2667 x 0 with resolution 15
     block, center = (0,7.375,0)
          size (46.7208,1.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (38,38,38)
     block, center = (0,5.375,0)
          size (46.7208,2.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (45,45,45)
     cylinder, center = (0,6.625,0)
          radius 0.75, height 1e+20, axis (0, 0, 1)
          dielectric constant epsilon diagonal = (60,60,60)
time for set_epsilon = 0.536426 s
time for set_conductivity = 0.014523 s
time for set_conductivity = 0.0144889 s
time for set_conductivity = 0.0147018 s
-----------
Meep progress: 43.56666666666666/507.6456362796297 = 8.6% done in 4.0s, 42.6s to go
on time step 1307 (time=43.5667), 0

run 0 finished at t = 507.6666666666667 (15230 timesteps)
-----------
Initializing structure...
time for choose_chunkdivision = 0.000519991 s
Working in 2D dimensions.
Computational cell is 46.7333 x 25.2667 x 0 with resolution 15
     block, center = (0,7.375,0)
          size (46.7208,1.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (38,38,38)
     block, center = (0,5.375,0)
          size (46.7208,2.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (45,45,45)
time for set_epsilon = 0.521684 s
time for set_conductivity = 0.01455 s
time for set_conductivity = 0.0148549 s
time for set_conductivity = 0.0137439 s
-----------
Meep progress: 43.2/507.6456362796297 = 8.5% done in 4.0s, 43.0s to go
on time step 1296 (time=43.2), 0.00308753 s/step
Meep progress: 86.33333333333333/507.6456362796297 = 17.0% done in 8.0s, 39.1s to go
on time step 2590 (time=86.3333), 0.00309269 s/step
Meep progr

run 0 finished at t = 507.6666666666667 (15230 timesteps)
-----------
Initializing structure...
time for choose_chunkdivision = 0.000497103 s
Working in 2D dimensions.
Computational cell is 46.7333 x 25.2667 x 0 with resolution 15
     block, center = (0,7.375,0)
          size (46.7208,1.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (38,38,38)
     block, center = (0,5.375,0)
          size (46.7208,2.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (45,45,45)
     cylinder, center = (0,6.625,0)
          radius 0.75, height 1e+20, axis (0, 0, 1)
          dielectric constant epsilon diagonal = (60,60,60)
time for set_epsilon = 0.536447 s
time for set_conductivity = 0.020534 s
time for set_conductivity = 0.0187819 s
time for set_conductivity = 0.0169549 s
-----------
Meep progress: 45.233333333333334/507.6456362796297 = 8.9% done in 4.0s, 40.9s to go
on time step 1357 (time=45.2333),

In [ ]:
ls